# School Dataset (Python)

import pandas as pd
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
#import janitor

import numpy as np

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.gofplots import ProbPlot

import warnings
warnings.filterwarnings("ignore")

## Data Sources
Ofsted Ratings (data/schools/brighton/OFSR_hist_long.csv): manually compiled referencing https://reports.ofsted.gov.uk/ and data/schools/brighton/2018-2019/846_school_information which is downloadable from DfE archives
Free School Meals (data/schools/brighton/fsm)
KS2 Results (Pri) (data/schools/brighton/ks2)
KS4 Results (Sec) (data/schools/brighton/ks4)
Pupil Destinations (data/schools/brighton/pupdest)

## Output
Running this ipynb gives ks2_hist_df.csv and ks4_hist_df.csv (in data/schools/brighton) which contains ratings for primary and secondary schools which will be needed for later geographical analysis. 

# Building up the school performance dataset: Primary Schools 2003-2019

To build up the historical school performance dataset, we will build the dataset using the first 2003 dataset (2000 for secondary school gcses), and left join each subsequent year's ratings to the dataset. This will allow us to account for changes in primary schools (opening/ closing) for each year. 

Starting with the 2003 dataset and repeated for each subsequent year
1. Read in england dataset for the year
2. Extract Brighton schools and key columns
3. Create new column LAESTAB that combines LEA/LA and ESTAB number (using regex)
4. Rename columns 
5. Use variables to sort and rank schools
6. Drop any empty schools
7. Left join yearly rankings to main dataframe
8. Save csv

For ranking, 2003-2006 is KS2, 2007-2011 is KS2 and OFSR, 2012-2019 is KS2, OFSR and FSMP

## 2003-2006 KS2

In [2]:
#read in xls
england_ks2_03 = pd.read_excel("schools/brighton/ks2/2002-2003_england_ks2final.xls", sheet_name=0)
#extract brighton schools
brighton_ks2_03 = england_ks2_03[england_ks2_03['LEA number'] == 846]
#extract key columns
ks2_2003 = brighton_ks2_03[['LEA number','Establishment number','School name','School postcode',
                                      'Percentage of pupils achieving level 5 in English','Percentage of pupils achieving level 5 in mathematics',
                                      'Percentage of pupils achieving level 5 in science','School VA measure']]
# checktypes = brighton_ks2_03_key.dtypes
# print(checktypes)

ks2_2003['LAESTAB'] = ks2_2003['LEA number'].astype(str) + ks2_2003['Establishment number'].astype(str)
ks2_2003.rename(columns={"Percentage of pupils achieving level 5 in English":"level5_english_p_03"},inplace=True)
ks2_2003.rename(columns={"Percentage of pupils achieving level 5 in mathematics":"level5_maths_p_03"},inplace=True)
ks2_2003.rename(columns={"Percentage of pupils achieving level 5 in science":"level5_science_p_03"},inplace=True)

#using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
ks2_2003['english_rank'] = ks2_2003['level5_english_p_03'].rank(ascending=False)
ks2_2003['maths_rank'] = ks2_2003['level5_maths_p_03'].rank(ascending=False)
ks2_2003['science_rank'] = ks2_2003['level5_science_p_03'].rank(ascending=False)

#we assign equal weights to each subject
#weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}

#Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up.
ks2_2003['rank'] = (ks2_2003['english_rank'] * 1/3 +
              ks2_2003['maths_rank'] * 1/3 +
              ks2_2003['science_rank'] * 1/3)

#using ['rank'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor) ['rank_band']
#we can then focus on the different types of schools for each year in the regression
ks2_2003.sort_values('rank', ascending=True, inplace=True)

bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2003['rank_band'] = pd.cut(ks2_2003['rank'], bins=bins, labels=labels)
#ks2_2003[['School name', 'rank_band']]


ks2_2003.head()


,LEA number,Establishment number,School name,School postcode,level5_english_p_03,level5_maths_p_03,level5_science_p_03,School VA measure,LAESTAB,english_rank,maths_rank,science_rank,rank,rank_band
7118,846,3318,St Bernadette's Catholic Primary School,BN1 6UT,0.59,0.44,0.81,101.2,8463318,1.0,6.0,1.0,2.666667,Outstanding
7111,846,3316,Our Lady of Lourdes RC School,BN2 7HA,0.45,0.59,0.69,101.5,8463316,5.0,1.0,5.0,3.666667,Outstanding
7094,846,2000,Balfour Junior School,BN1 6NE,0.41,0.51,0.73,99.1,8462000,7.5,4.0,3.0,4.833333,Outstanding
7131,846,2028,Stanford Junior School,BN1 5PR,0.43,0.49,0.71,101.6,8462028,6.0,5.0,4.0,5.000000,Outstanding
7134,846,2036,Westdene Primary School,BN1 5GN,0.52,0.42,0.58,100.8,8462036,2.5,7.5,8.0,6.000000,Outstanding


In [3]:
# 44 pri schools in 2003
#building dataframe
ks2_hist_df = ks2_2003[['LAESTAB','School name', 'School postcode', 'rank_band']]
ks2_hist_df.rename(columns={'rank_band':"2003"}, inplace=True)
ks2_hist_df.head()

,LAESTAB,School name,School postcode,2003
7118,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding
7111,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding
7094,8462000,Balfour Junior School,BN1 6NE,Outstanding
7131,8462028,Stanford Junior School,BN1 5PR,Outstanding
7134,8462036,Westdene Primary School,BN1 5GN,Outstanding


In [4]:
# 2004
#read in xls
england_ks2_04 = pd.read_excel("schools/brighton/ks2/2003-2004_england_ks2final.xlsx", sheet_name=3, header=1)
#extract brighton schools
brighton_ks2_04 = england_ks2_04[england_ks2_04['LEA number']== 846]
#extract key columns
ks2_2004 = brighton_ks2_04[['LEA number','Establishment number','School name','School postcode',
                                      'Percentage of pupils achieving level 5 in English','Percentage of pupils achieving level 5 in mathematics',
                                      'Percentage of pupils achieving level 5 in science','School VA measure']]
ks2_2004['LAESTAB'] = ks2_2004['LEA number'].astype(str) + ks2_2004['Establishment number'].astype(str)
ks2_2004.rename(columns={"Percentage of pupils achieving level 5 in English":"level5_english_p_04"},inplace=True)
ks2_2004.rename(columns={"Percentage of pupils achieving level 5 in mathematics":"level5_maths_p_04"},inplace=True)
ks2_2004.rename(columns={"Percentage of pupils achieving level 5 in science":"level5_science_p_04"},inplace=True)

#using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
ks2_2004['english_rank'] = ks2_2004['level5_english_p_04'].rank(ascending=False)
ks2_2004['maths_rank'] = ks2_2004['level5_maths_p_04'].rank(ascending=False)
ks2_2004['science_rank'] = ks2_2004['level5_science_p_04'].rank(ascending=False)

#we assign equal weights to each subject
#weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}

#Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up.
ks2_2004['rank'] = (ks2_2004['english_rank'] * 1/3 +
              ks2_2004['maths_rank'] * 1/3 +
              ks2_2004['science_rank'] * 1/3)
#using ['rank'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor) ['rank_band']
#we can then focus on the different types of schools for each year in the regression
ks2_2004.sort_values('rank', ascending=True, inplace=True)

ks2_2004['rank_band'] = pd.cut(ks2_2004['rank'], bins=bins, labels=labels)


ks2_2004.head()

,LEA number,Establishment number,School name,School postcode,level5_english_p_04,level5_maths_p_04,level5_science_p_04,School VA measure,LAESTAB,english_rank,maths_rank,science_rank,rank,rank_band
7218,846,2156,Queen's Park Primary School,BN2 0BN,0.45,0.59,0.84,102.2,8462156,4.0,2.0,1.0,2.333333,Outstanding
7198,846,2000,Balfour Junior School,BN1 6NE,0.5,0.45,0.73,100.6,8462000,1.5,6.0,3.0,3.500000,Outstanding
7215,846,3316,Our Lady of Lourdes RC School,BN2 7HA,0.36,0.57,0.75,101.7,8463316,8.0,3.0,2.0,4.333333,Outstanding
7222,846,3318,St Bernadette's Catholic Primary School,BN1 6UT,0.48,0.63,0.59,101,8463318,3.0,1.0,9.5,4.500000,Outstanding
7238,846,2036,Westdene Primary School,BN1 5GN,0.41,0.51,0.67,101.5,8462036,6.0,4.0,5.5,5.166667,Outstanding


In [5]:
# 44 pri schools in 2004 (same as 2003)
#building dataframe
ks2_2004_add = ks2_2004[['LAESTAB','School name', 'School postcode', 'rank_band']]
ks2_2004_add.rename(columns={'rank_band':"2004"}, inplace=True)

#left join to ks2_hist_df
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2004_add[['LAESTAB','2004']], how='left', on='LAESTAB')
ks2_hist_df.head(10)

,LAESTAB,School name,School postcode,2003,2004
0,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding,Outstanding
1,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding,Outstanding
2,8462000,Balfour Junior School,BN1 6NE,Outstanding,Outstanding
3,8462028,Stanford Junior School,BN1 5PR,Outstanding,Outstanding
4,8462036,Westdene Primary School,BN1 5GN,Outstanding,Outstanding
5,8462156,Queen's Park Primary School,BN2 0BN,Outstanding,Outstanding
6,8463341,Cottesmore St Mary's Catholic Primary School,BN3 6NB,Outstanding,Satisfactory
7,8463344,St Mary's Catholic Primary School,BN41 1LB,Outstanding,Satisfactory
8,8462158,Woodingdean Primary School,BN2 6BB,Outstanding,Outstanding
9,8463328,Aldrington CofE Primary School,BN3 7QD,Good,Outstanding


In [6]:
# 2005
#read in xls
england_ks2_05 = pd.read_excel("schools/brighton/ks2/2004-2005_england_ks2final.xlsx", sheet_name=0)
#extract brighton schools
brighton_ks2_05 = england_ks2_05[england_ks2_05['LEA number']== 846]
#extract key columns
ks2_2005 = brighton_ks2_05[['LEA number','URN','Establishment number','School name','School postcode',
                                      'Percentage of pupils achieving Level 5 in English','Percentage of pupils achieving Level 5 in mathematics',
                                      'Percentage of pupils achieving Level 5 in science','School VA measure']]
ks2_2005['LAESTAB'] = ks2_2005['LEA number'].astype(str) + ks2_2005['Establishment number'].astype(str)
ks2_2005.rename(columns={"Percentage of pupils achieving Level 5 in English":"level5_english_p_05"},inplace=True)
ks2_2005.rename(columns={"Percentage of pupils achieving Level 5 in mathematics":"level5_maths_p_05"},inplace=True)
ks2_2005.rename(columns={"Percentage of pupils achieving Level 5 in science":"level5_science_p_05"},inplace=True)

#using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
ks2_2005['english_rank'] = ks2_2005['level5_english_p_05'].rank(ascending=False)
ks2_2005['maths_rank'] = ks2_2005['level5_maths_p_05'].rank(ascending=False)
ks2_2005['science_rank'] = ks2_2005['level5_science_p_05'].rank(ascending=False)
#we assign equal weights to each subject
#weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
#Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up.
ks2_2005['rank'] = (ks2_2005['english_rank'] * 1/3 +
              ks2_2005['maths_rank'] * 1/3 +
              ks2_2005['science_rank'] * 1/3)
#using ['rank'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor) ['rank_band']
#we can then focus on the different types of schools for each year in the regression
ks2_2005.sort_values('rank', ascending=True, inplace=True)
ks2_2005['rank_band'] = pd.cut(ks2_2005['rank'], bins=bins, labels=labels)
ks2_2005.head()

,LEA number,URN,Establishment number,School name,School postcode,level5_english_p_05,level5_maths_p_05,level5_science_p_05,School VA measure,LAESTAB,english_rank,maths_rank,science_rank,rank,rank_band
7171,846,114361,2000,Balfour Junior School,BN1 6NE,0.42,0.57,0.79,101.3,8462000,4.0,2.0,2.0,2.666667,Outstanding
7207,846,114544,3316,Our Lady of Lourdes RC School,BN2 7HA,0.41,0.63,0.78,102.2,8463316,6.0,1.0,3.5,3.500000,Outstanding
7181,846,114380,2036,Westdene Primary School,BN1 5GN,0.5,0.53,0.67,102.1,8462036,1.5,4.0,8.0,4.500000,Outstanding
7209,846,114546,3318,St Bernadette's Catholic Primary School,BN1 6UT,0.45,0.45,0.76,101.2,8463318,3.0,7.0,5.0,5.000000,Outstanding
7211,846,114556,3329,St Andrew's CofE (Aided) Primary School,BN3 3YT,0.5,0.47,0.61,101.4,8463329,1.5,5.0,11.0,5.833333,Outstanding


In [7]:
# 44 pri schools in 2005 (same as 2003-4)
#building dataframe
ks2_2005_add = ks2_2005[['LAESTAB','School name', 'School postcode', 'rank_band']]
ks2_2005_add.rename(columns={'rank_band':"2005"}, inplace=True)

#left join to ks2_hist_df
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2005_add[['LAESTAB','2005']], how='left', on='LAESTAB')
ks2_hist_df.head(10)

,LAESTAB,School name,School postcode,2003,2004,2005
0,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding,Outstanding,Outstanding
1,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding,Outstanding,Outstanding
2,8462000,Balfour Junior School,BN1 6NE,Outstanding,Outstanding,Outstanding
3,8462028,Stanford Junior School,BN1 5PR,Outstanding,Outstanding,Outstanding
4,8462036,Westdene Primary School,BN1 5GN,Outstanding,Outstanding,Outstanding
5,8462156,Queen's Park Primary School,BN2 0BN,Outstanding,Outstanding,Outstanding
6,8463341,Cottesmore St Mary's Catholic Primary School,BN3 6NB,Outstanding,Satisfactory,Good
7,8463344,St Mary's Catholic Primary School,BN41 1LB,Outstanding,Satisfactory,Satisfactory
8,8462158,Woodingdean Primary School,BN2 6BB,Outstanding,Outstanding,Outstanding
9,8463328,Aldrington CofE Primary School,BN3 7QD,Good,Outstanding,Good


In [8]:
# 2006
#read in xls
england_ks2_06 = pd.read_excel("schools/brighton/ks2/2005-2006_england_ks2final.xls", sheet_name=0, header=1)
#extract brighton schools
brighton_ks2_06 = england_ks2_06[england_ks2_06['LA number']== 846]
#extract key columns
ks2_2006 = brighton_ks2_06[['LA number','Establishment number','School name','School postcode',
                                      'Percentage of pupils achieving Level 5 in English','Percentage of pupils achieving Level 5 in mathematics',
                                      'Percentage of pupils achieving Level 5 in science','School VA measure']]
ks2_2006['LAESTAB'] = ks2_2006['LA number'].astype(str) + ks2_2006['Establishment number'].astype(str)
ks2_2006.rename(columns={"Percentage of pupils achieving Level 5 in English":"level5_english_p_06"},inplace=True)
ks2_2006.rename(columns={"Percentage of pupils achieving Level 5 in mathematics":"level5_maths_p_06"},inplace=True)
ks2_2006.rename(columns={"Percentage of pupils achieving Level 5 in science":"level5_science_p_06"},inplace=True)

ks2_2006.drop(ks2_2006[(ks2_2006['LAESTAB'] == '8463314')].index, inplace=True)

#using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
ks2_2006['english_rank'] = ks2_2006['level5_english_p_06'].rank(ascending=False)
ks2_2006['maths_rank'] = ks2_2006['level5_maths_p_06'].rank(ascending=False)
ks2_2006['science_rank'] = ks2_2006['level5_science_p_06'].rank(ascending=False)
#we assign equal weights to each subject
#weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
#Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up.
ks2_2006['rank'] = (ks2_2006['english_rank'] * 1/3 +
              ks2_2006['maths_rank'] * 1/3 +
              ks2_2006['science_rank'] * 1/3)

#using ['rank'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor) ['rank_band']
#we can then focus on the different types of schools for each year in the regression
ks2_2006.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2006['rank_band'] = pd.cut(ks2_2006['rank'], bins=bins, labels=labels)
ks2_2006

,LA number,Establishment number,School name,School postcode,level5_english_p_06,level5_maths_p_06,level5_science_p_06,School VA measure,LAESTAB,english_rank,maths_rank,science_rank,rank,rank_band
7146,846,3341,Cottesmore St Mary's Catholic Primary School,BN3 6NB,0.6,0.57,0.68,99.7,8463341,2.0,2.0,5.5,3.166667,Outstanding
7163,846,3318,St Bernadette's Catholic Primary School,BN1 6UT,0.57,0.45,0.68,100.6,8463318,3.0,7.5,5.5,5.333333,Outstanding
7159,846,2156,Queen's Park Primary School,BN2 2BN,0.41,0.56,0.95,101.3,8462156,15.0,3.0,1.0,6.333333,Outstanding
7139,846,2000,Balfour Junior School,BN1 6NE,0.49,0.45,0.8,99.9,8462000,10.0,7.5,3.0,6.833333,Outstanding
7173,846,3315,St Paul's CofE Primary School and Nursery,BN1 3LP,0.35,0.65,0.88,101.6,8463315,21.0,1.0,2.0,8.000000,Outstanding
7179,846,2036,Westdene Primary School,BN1 5GN,0.45,0.47,0.65,100.9,8462036,11.0,5.5,8.5,8.333333,Outstanding
7161,846,3329,St Andrew's CofE (Aided) Primary School,BN3 3YT,0.5,0.47,0.62,101,8463329,8.5,5.5,11.5,8.500000,Outstanding
7167,846,3304,"St Margaret's CofE Primary School, Rottingdean",BN2 7HB,0.53,0.44,0.56,101.1,8463304,4.5,9.5,16.0,10.000000,Outstanding
7156,846,3316,Our Lady of Lourdes RC School,BN2 7HA,0.38,0.48,0.66,100.7,8463316,19.5,4.0,7.0,10.166667,Outstanding
7148,846,2155,Elm Grove Primary School,BN2 3ES,0.61,0.32,0.63,101.3,8462155,1.0,19.5,10.0,10.166667,Outstanding


In [9]:
# 43 pri schools in 2006, due to dropping of St Josephs with small cohort (unlike 2003-5)
#building dataframe
ks2_2006_add = ks2_2006[['LAESTAB','School name', 'School postcode', 'rank_band']]
ks2_2006_add.rename(columns={'rank_band':"2006"}, inplace=True)

#left join to ks2_hist_df
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2006_add[['LAESTAB','2006']], how='left', on='LAESTAB')
ks2_hist_df.head(10)

,LAESTAB,School name,School postcode,2003,2004,2005,2006
0,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding,Outstanding,Outstanding,Outstanding
1,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding,Outstanding,Outstanding,Outstanding
2,8462000,Balfour Junior School,BN1 6NE,Outstanding,Outstanding,Outstanding,Outstanding
3,8462028,Stanford Junior School,BN1 5PR,Outstanding,Outstanding,Outstanding,Outstanding
4,8462036,Westdene Primary School,BN1 5GN,Outstanding,Outstanding,Outstanding,Outstanding
5,8462156,Queen's Park Primary School,BN2 0BN,Outstanding,Outstanding,Outstanding,Outstanding
6,8463341,Cottesmore St Mary's Catholic Primary School,BN3 6NB,Outstanding,Satisfactory,Good,Outstanding
7,8463344,St Mary's Catholic Primary School,BN41 1LB,Outstanding,Satisfactory,Satisfactory,Satisfactory
8,8462158,Woodingdean Primary School,BN2 6BB,Outstanding,Outstanding,Outstanding,Good
9,8463328,Aldrington CofE Primary School,BN3 7QD,Good,Outstanding,Good,Outstanding


In [10]:
#ks2_hist_df.to_csv('schools/brighton/2002-2006/historical_2003-2006_brighton_ks2final.csv', index=False)
#saving in case

## 2007-2019 (KS2 and OFSR, then add FSM for 2012 onwards)

### 2007 manual

In [3]:
#OFSTED Rankings
ofsr_long = pd.read_csv("schools/brighton/OFSR_hist_long.csv")

#year_list = ofsr_long['YEAR'].unique()
ofsr_wide = ofsr_long.pivot(index='SCHNAME', columns='YEAR', values='OFSR')
ofsr_wide.reset_index(inplace=True)
ofsr_wide.columns.tolist()

['SCHNAME',
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2014,
 2015,
 2016,
 2017,
 2018,
 2019,
 2020,
 2021,
 2022]

In [12]:
# # 2007
# #read in xls
# england_ks2_07 = pd.read_excel("schools/brighton/ks2/2006-2007_england_ks2final.xls", sheet_name=0, header=1)
# #extract brighton schools
# brighton_ks2_07 = england_ks2_07[england_ks2_07['LA number']== 846]
# #extract key columns
# ks2_2007 = brighton_ks2_07[['LA number','Establishment number','School name','School postcode',
#                                       'Percentage of pupils achieving Level 5 in English','Percentage of pupils achieving Level 5 in mathematics',
#                                       'Percentage of pupils achieving Level 5 in science','CVA measure for pupils at the end of KS2']]
# ks2_2007['LAESTAB'] = ks2_2007['LA number'].astype(str) + ks2_2007['Establishment number'].astype(str)
# ks2_2007.rename(columns={"Percentage of pupils achieving Level 5 in English":"level5_english_p_07"},inplace=True)
# ks2_2007.rename(columns={"Percentage of pupils achieving Level 5 in mathematics":"level5_maths_p_07"},inplace=True)
# ks2_2007.rename(columns={"Percentage of pupils achieving Level 5 in science":"level5_science_p_07"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2007['english_rank'] = ks2_2007['level5_english_p_07'].rank(ascending=False)
# ks2_2007['maths_rank'] = ks2_2007['level5_maths_p_07'].rank(ascending=False)
# ks2_2007['science_rank'] = ks2_2007['level5_science_p_07'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2007['acad_rank'] = (ks2_2007['english_rank'] * 1/3 +
#               ks2_2007['maths_rank'] * 1/3 +
#               ks2_2007['science_rank'] * 1/3)

# #merge 2007 ks2 acad results with 2007 ofsted rating
# ks2_2007 = pd.merge(ks2_2007, ofsr_wide[['SCHNAME',2007]], how='left', left_on='School name', right_on='SCHNAME')
# ks2_2007.rename(columns={2007:"ofsr"}, inplace=True)

# manually input 
# ks2_2007.to_csv('schools/brighton/2007-2011/2006-2007_brighton_ks2_missing.csv', index=False)

ks2_2007 = pd.read_csv('schools/brighton/2007-2011/2006-2007_brighton_completed.csv')
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Inadequate":4}
ks2_2007['ofsr_values'] = ks2_2007['ofsr'].map(mapping)
#ks2_2007['ofsr_band'] = pd.cut(ks2_2007['ofsr'], bins=4)

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2007['rank'] = (ks2_2007['acad_rank'].rank(ascending=True) * 0.5 +
                    ks2_2007['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
#we can then focus on the different types of schools for each year in the regression
ks2_2007.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2007['rank_band'] = pd.cut(ks2_2007['rank'], bins=bins, labels=labels)

In [13]:
# 44 pri schools in 2007, just like 2003-5 (43 in 2006)
#building dataframe
ks2_2007_add = ks2_2007[['LAESTAB','School name', 'School postcode', 'rank_band']]
ks2_2007_add.rename(columns={'rank_band':"2007"}, inplace=True)
ks2_2007_add["LAESTAB"] = ks2_2007_add["LAESTAB"].astype(str)
#left join to ks2_hist_df
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2007_add[['LAESTAB','2007']], how='left', on='LAESTAB')
ks2_hist_df.head(10)

,LAESTAB,School name,School postcode,2003,2004,2005,2006,2007
0,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
1,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding,Outstanding,Outstanding,Outstanding,Good
2,8462000,Balfour Junior School,BN1 6NE,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
3,8462028,Stanford Junior School,BN1 5PR,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
4,8462036,Westdene Primary School,BN1 5GN,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
5,8462156,Queen's Park Primary School,BN2 0BN,Outstanding,Outstanding,Outstanding,Outstanding,Satisfactory
6,8463341,Cottesmore St Mary's Catholic Primary School,BN3 6NB,Outstanding,Satisfactory,Good,Outstanding,Outstanding
7,8463344,St Mary's Catholic Primary School,BN41 1LB,Outstanding,Satisfactory,Satisfactory,Satisfactory,Satisfactory
8,8462158,Woodingdean Primary School,BN2 6BB,Outstanding,Outstanding,Outstanding,Good,Good
9,8463328,Aldrington CofE Primary School,BN3 7QD,Good,Outstanding,Good,Outstanding,Outstanding


### 2008-2019

In [14]:
# # 2008
# #read in xls
# england_ks2_08 = pd.read_excel("schools/brighton/ks2/2007-2008_england_ks2final.xls", sheet_name=2, header=1)
# #extract brighton schools
# brighton_ks2_08 = england_ks2_08[england_ks2_08['LA number']== 846]
# #extract key columns
# ks2_2008 = brighton_ks2_08[['LA number','Establishment number','School name','School postcode',
#                                       'Percentage of pupils achieving Level 5 in English','Percentage of pupils achieving Level 5 in mathematics',
#                                       'Percentage of pupils achieving Level 5 in science','CVA measure for pupils at the end of KS2']]
# ks2_2008['LAESTAB'] = ks2_2008['LA number'].astype(str) + ks2_2008['Establishment number'].astype(str)
# ks2_2008.rename(columns={"Percentage of pupils achieving Level 5 in English":"level5_english_p_08"},inplace=True)
# ks2_2008.rename(columns={"Percentage of pupils achieving Level 5 in mathematics":"level5_maths_p_08"},inplace=True)
# ks2_2008.rename(columns={"Percentage of pupils achieving Level 5 in science":"level5_science_p_08"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2008['english_rank'] = ks2_2008['level5_english_p_08'].rank(ascending=False)
# ks2_2008['maths_rank'] = ks2_2008['level5_maths_p_08'].rank(ascending=False)
# ks2_2008['science_rank'] = ks2_2008['level5_science_p_08'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2008['acad_rank'] = (ks2_2008['english_rank'] * 1/3 +
#               ks2_2008['maths_rank'] * 1/3 +
#               ks2_2008['science_rank'] * 1/3)

# #merge 2007 ks2 acad results with 2007 ofsted rating
# ks2_2008 = pd.merge(ks2_2008, ofsr_wide[['SCHNAME',2008]], how='left', left_on='School name', right_on='SCHNAME')
# ks2_2008.rename(columns={2008:"ofsr"}, inplace=True)

# #manually input 
# ks2_2008.to_csv('schools/brighton/2007-2011/2007-2008_brighton_ks2_missing.csv', index=False)

ks2_2008 = pd.read_csv('schools/brighton/2007-2011/2007-2008_brighton_ks2_completed.csv')
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Inadequate":4}
ks2_2008['ofsr_values'] = ks2_2008['ofsr'].map(mapping)

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2008['rank'] = (ks2_2008['acad_rank'].rank(ascending=True) * 0.5 +
                    ks2_2008['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
#we can then focus on the different types of schools for each year in the regression
ks2_2008.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2008['rank_band'] = pd.cut(ks2_2008['rank'], bins=bins, labels=labels)

In [15]:
# 43 pri schools in 2008. 44 schools in 2003-5, 07. (43 in 06, 08)
#building dataframe
ks2_2008_add = ks2_2008[['LAESTAB','School name', 'School postcode', 'rank_band']]
ks2_2008_add.rename(columns={'rank_band':"2008"}, inplace=True)
ks2_2008_add["LAESTAB"] = ks2_2008_add["LAESTAB"].astype(str)
#left join to ks2_hist_df
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2008_add[['LAESTAB','2008']], how='left', on='LAESTAB')
ks2_hist_df.head(10)

,LAESTAB,School name,School postcode,2003,2004,2005,2006,2007,2008
0,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
1,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding,Outstanding,Outstanding,Outstanding,Good,Good
2,8462000,Balfour Junior School,BN1 6NE,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
3,8462028,Stanford Junior School,BN1 5PR,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
4,8462036,Westdene Primary School,BN1 5GN,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
5,8462156,Queen's Park Primary School,BN2 0BN,Outstanding,Outstanding,Outstanding,Outstanding,Satisfactory,Outstanding
6,8463341,Cottesmore St Mary's Catholic Primary School,BN3 6NB,Outstanding,Satisfactory,Good,Outstanding,Outstanding,Outstanding
7,8463344,St Mary's Catholic Primary School,BN41 1LB,Outstanding,Satisfactory,Satisfactory,Satisfactory,Satisfactory,Satisfactory
8,8462158,Woodingdean Primary School,BN2 6BB,Outstanding,Outstanding,Outstanding,Good,Good,Good
9,8463328,Aldrington CofE Primary School,BN3 7QD,Good,Outstanding,Good,Outstanding,Outstanding,Outstanding


In [16]:
#west blatch changed to 846 2093 in 2009 instead of 2068 prior

In [17]:
# # 2009
# #read in xls
# england_ks2_09 = pd.read_excel("schools/brighton/ks2/2008-2009_england_ks2final.xls", sheet_name=1, header=1)
# #extract brighton schools
# brighton_ks2_09 = england_ks2_09[england_ks2_09['LA number']== 846]
# #extract key columns
# ks2_2009 = brighton_ks2_09[['LA number','Establishment number','School name','School postcode',
#                                       'Percentage of pupils achieving Level 5 in English','Percentage of pupils achieving Level 5 in mathematics',
#                                       'Percentage of pupils achieving Level 5 in science','CVA measure for pupils at the end of KS2']]
# ks2_2009['LAESTAB'] = ks2_2009['LA number'].astype(str) + ks2_2009['Establishment number'].astype(str)
# ks2_2009.rename(columns={"Percentage of pupils achieving Level 5 in English":"level5_english_p_09"},inplace=True)
# ks2_2009.rename(columns={"Percentage of pupils achieving Level 5 in mathematics":"level5_maths_p_09"},inplace=True)
# ks2_2009.rename(columns={"Percentage of pupils achieving Level 5 in science":"level5_science_p_09"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2009['english_rank'] = ks2_2009['level5_english_p_09'].rank(ascending=False)
# ks2_2009['maths_rank'] = ks2_2009['level5_maths_p_09'].rank(ascending=False)
# ks2_2009['science_rank'] = ks2_2009['level5_science_p_09'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2009['acad_rank'] = (ks2_2009['english_rank'] * 1/3 +
#               ks2_2009['maths_rank'] * 1/3 +
#               ks2_2009['science_rank'] * 1/3)

# #merge 2007 ks2 acad results with 2007 ofsted rating
# ks2_2009 = pd.merge(ks2_2009, ofsr_wide[['SCHNAME',2009]], how='left', left_on='School name', right_on='SCHNAME')
# ks2_2009.rename(columns={2009:"ofsr"}, inplace=True)

# #manually input 
# ks2_2009.to_csv('schools/brighton/2007-2011/2008-2009_brighton_ks2_missing.csv', index=False)

ks2_2009 = pd.read_csv('schools/brighton/2007-2011/2008-2009_brighton_ks2_completed.csv')

mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Inadequate":4}
ks2_2009['ofsr_values'] = ks2_2009['ofsr'].map(mapping)

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2009['rank'] = (ks2_2009['acad_rank'].rank(ascending=True) * 0.5 +
                    ks2_2009['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
#we can then focus on the different types of schools for each year in the regression
ks2_2009.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2009['rank_band'] = pd.cut(ks2_2009['rank'], bins=bins, labels=labels)

In [18]:
# 44 pri schools in 2009. 44 schools in 2003-5, 07. (43 in 06, 08)
#building dataframe
ks2_2009_add = ks2_2009[['LAESTAB','School name', 'School postcode', 'rank_band']]
ks2_2009_add.rename(columns={'rank_band':"2009"}, inplace=True)
ks2_2009_add["LAESTAB"] = ks2_2009_add["LAESTAB"].astype(str)
#left join to ks2_hist_df
#ks2_hist_df['2008'].fillna('Satisfactory') manually fill later
ks2_hist_df['LAESTAB'].replace(to_replace='8462068', value='8462093', inplace=True)
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2009_add[['LAESTAB','2009']], how='left', on='LAESTAB')
ks2_hist_df.head(10)

,LAESTAB,School name,School postcode,2003,2004,2005,2006,2007,2008,2009
0,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
1,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding,Outstanding,Outstanding,Outstanding,Good,Good,Good
2,8462000,Balfour Junior School,BN1 6NE,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
3,8462028,Stanford Junior School,BN1 5PR,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
4,8462036,Westdene Primary School,BN1 5GN,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
5,8462156,Queen's Park Primary School,BN2 0BN,Outstanding,Outstanding,Outstanding,Outstanding,Satisfactory,Outstanding,Satisfactory
6,8463341,Cottesmore St Mary's Catholic Primary School,BN3 6NB,Outstanding,Satisfactory,Good,Outstanding,Outstanding,Outstanding,Outstanding
7,8463344,St Mary's Catholic Primary School,BN41 1LB,Outstanding,Satisfactory,Satisfactory,Satisfactory,Satisfactory,Satisfactory,Satisfactory
8,8462158,Woodingdean Primary School,BN2 6BB,Outstanding,Outstanding,Outstanding,Good,Good,Good,Good
9,8463328,Aldrington CofE Primary School,BN3 7QD,Good,Outstanding,Good,Outstanding,Outstanding,Outstanding,Outstanding


In [19]:
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)
#manually input West Blatch Poor for 2009,

In [20]:
## # 2010
# #read in xls
# england_ks2_10 = pd.read_excel("schools/brighton/ks2/2009-2010_england_ks2final.xls", sheet_name=0, header=1)
# #extract brighton schools
# brighton_ks2_10 = england_ks2_10[england_ks2_10['LaCode']== 846]

# #drop SchoolName == 'Brighton and Hove'
# brighton_ks2_10.drop(brighton_ks2_10[(brighton_ks2_10['SchoolName'] == 'Brighton and Hove')].index, inplace=True)
# #extract key columns
# ks2_2010 = brighton_ks2_10[['LaCode','EstabCode','SchoolName','PostCode',
#                                       'PEngTaL5','PMatTaL5',
#                                       'PSciTaL5','CvaMea']]

# #drop LT11 or NaN for academic subjects, as these are special schools
# ks2_2010.drop(ks2_2010[(ks2_2010['PEngTaL5'] == 'LT11')].index, inplace=True)
# ks2_2010 = ks2_2010.dropna(subset=["PEngTaL5"])
# #change subjs to float
# #change la estab to int
# ks2_2010['LAESTAB'] = ks2_2010['LaCode'].astype(int).astype(str) + ks2_2010['EstabCode'].astype(int).astype(str)
# ks2_2010['PEngTaL5'].astype(float)
# ks2_2010['PMatTaL5'].astype(float)
# ks2_2010['PSciTaL5'].astype(float)

# ks2_2010.rename(columns={"PEngTaL5":"level5_english_p_10"},inplace=True)
# ks2_2010.rename(columns={"PMatTaL5":"level5_maths_p_10"},inplace=True)
# ks2_2010.rename(columns={"PSciTaL5":"level5_science_p_10"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2010['english_rank'] = ks2_2010['level5_english_p_10'].rank(ascending=False)
# ks2_2010['maths_rank'] = ks2_2010['level5_maths_p_10'].rank(ascending=False)
# ks2_2010['science_rank'] = ks2_2010['level5_science_p_10'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2010['acad_rank'] = (ks2_2010['english_rank'] * 1/3 +
#               ks2_2010['maths_rank'] * 1/3 +
#               ks2_2010['science_rank'] * 1/3)

# #merge 2007 ks2 acad results with 2007 ofsted rating
# ks2_2010 = pd.merge(ks2_2010, ofsr_wide[['SCHNAME',2010]], how='left', left_on='SchoolName', right_on='SCHNAME')
# ks2_2010.rename(columns={2010:"ofsr"}, inplace=True)

# #manually input 
# ks2_2010.to_csv('schools/brighton/2007-2011/2009-2010_brighton_ks2_missing.csv', index=False)

ks2_2010 = pd.read_csv('schools/brighton/2007-2011/2009-2010_brighton_ks2_completed.csv')
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Inadequate":4}
ks2_2010['ofsr_values'] = ks2_2010['ofsr'].map(mapping)

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2010['rank'] = (ks2_2010['acad_rank'].rank(ascending=True) * 0.5 +
                    ks2_2010['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
#we can then focus on the different types of schools for each year in the regression
ks2_2010.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2010['rank_band'] = pd.cut(ks2_2010['rank'], bins=bins, labels=labels)

In [21]:
## 44 pri schools in 2010. 44 schools in 2003-5, 07,09. (43 in 06, 08)
#building dataframe
ks2_2010_add = ks2_2010[['LAESTAB','SchoolName', 'PostCode', 'rank_band']]
ks2_2010_add.rename(columns={'rank_band':"2010"}, inplace=True)
ks2_2010_add["LAESTAB"] = ks2_2010_add["LAESTAB"].astype(str)
#left join to ks2_hist_df
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2010_add[['LAESTAB','2010']], how='left', on='LAESTAB')
ks2_hist_df.head(10)
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)

In [22]:
# # 2011
# #read in csv
# brighton_ks2_11 = pd.read_csv("schools/brighton/2007-2011/2010-2011_846_ks2final.csv")

# brighton_ks2_11.drop(brighton_ks2_11[(brighton_ks2_11['SCHNAME'] == 'Brighton and Hove')].index, inplace=True)
# brighton_ks2_11.drop(brighton_ks2_11[(brighton_ks2_11['SCHNAME'] == 'NATIONAL')].index, inplace=True)
# #extract key columns
# ks2_2011 = brighton_ks2_11[['LEA','ESTAB','URN', 'SCHNAME','PCODE',
#                                       'PTENGAXTA','PTMATAXTA',
#                                       'PTSCIAXTA']]

# #drop LT11 or NaN for academic subjects, as these are special schools
# ks2_2011.drop(ks2_2011[(ks2_2011['PTENGAXTA'] == 'SUPP')].index, inplace=True)

# ks2_2011 = ks2_2011.dropna(subset=["PTENGAXTA"])
# #change subjs to float
# #change la estab to int
# ks2_2011['LEA'] = ks2_2011['LEA'].astype(int)
# ks2_2011['ESTAB'] = ks2_2011['ESTAB'].astype(int)
# ks2_2011['LAESTAB'] = ks2_2011['LEA'].astype(str) + ks2_2011['ESTAB'].astype(str)
# ks2_2011["PTENGAXTA"] = ks2_2011["PTENGAXTA"].str.replace("%", "").astype(int)
# ks2_2011["PTMATAXTA"] = ks2_2011["PTMATAXTA"].str.replace("%", "").astype(int)
# ks2_2011["PTSCIAXTA"] = ks2_2011["PTSCIAXTA"].str.replace("%", "").astype(int)

# ks2_2011.rename(columns={"PTENGAXTA":"level5_english_p_11"},inplace=True)
# ks2_2011.rename(columns={"PTMATAXTA":"level5_maths_p_11"},inplace=True)
# ks2_2011.rename(columns={"PTSCIAXTA":"level5_science_p_11"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2011['english_rank'] = ks2_2011['level5_english_p_11'].rank(ascending=False)
# ks2_2011['maths_rank'] = ks2_2011['level5_maths_p_11'].rank(ascending=False)
# ks2_2011['science_rank'] = ks2_2011['level5_science_p_11'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2011['acad_rank'] = (ks2_2011['english_rank'] * 1/3 +
#               ks2_2011['maths_rank'] * 1/3 +
#               ks2_2011['science_rank'] * 1/3)

# #merge 2007 ks2 acad results with 2007 ofsted rating
# ks2_2011 = pd.merge(ks2_2011, ofsr_wide[['SCHNAME',2011]], how='left', left_on='SCHNAME', right_on='SCHNAME')
# ks2_2011.rename(columns={2011:"ofsr"}, inplace=True)

# #manually input 
# ks2_2011.to_csv('schools/brighton/2007-2011/2010-2011_brighton_ks2_missing.csv', index=False)

ks2_2011 = pd.read_csv('schools/brighton/2007-2011/2010-2011_brighton_ks2_completed.csv')
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Inadequate":4}
ks2_2011['ofsr_values'] = ks2_2011['ofsr'].map(mapping)

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2011['rank'] = (ks2_2011['acad_rank'].rank(ascending=True) * 0.5 +
                    ks2_2011['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
#we can then focus on the different types of schools for each year in the regression
ks2_2011.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2011['rank_band'] = pd.cut(ks2_2011['rank'], bins=bins, labels=labels)

In [23]:
## 43 pri schools in 2011. 44 schools in 2003-5, 07,09. (43 in 06, 08,11)
#2011 balfour has no data, business as usual (Outstanding)
#building dataframe
ks2_2011_add = ks2_2011[['LAESTAB','SCHNAME', 'PCODE', 'rank_band']]
ks2_2011_add.rename(columns={'rank_band':"2011"}, inplace=True)
#ks2_2011_add["LAESTAB"] = ks2_2011_add["LAESTAB"].astype(int)
#left join to ks2_hist_df
ks2_hist_df = pd.read_csv('schools/brighton/ks2_hist_df.csv')
#ks2_hist_df["LAESTAB"] = ks2_hist_df["LAESTAB"].astype(str)
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2011_add[['LAESTAB','2011']], how='left', on='LAESTAB')
ks2_hist_df.head()
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)

In [15]:
#2012
# #read in csv
# brighton_ks2_12 = pd.read_csv("schools/brighton/2012-2019/2011-2012_846_ks2final.csv")

# #drop non-school rows
# brighton_ks2_12.drop(brighton_ks2_12[(brighton_ks2_12['SCHNAME'] == 'Brighton and Hove')].index, inplace=True)
# brighton_ks2_12.drop(brighton_ks2_12[(brighton_ks2_12['SCHNAME'] == 'NATIONAL')].index, inplace=True)
# brighton_ks2_12.drop(brighton_ks2_12[(brighton_ks2_12['NFTYPE'] == 'CYS')].index, inplace=True)

# #note Bilingual Pri School which has just started, but no data, its ofsted starts from 2014
# #extract key columns
# ks2_2012 = brighton_ks2_12[['LEA','ESTAB','URN', 'SCHNAME','PCODE',
#                                       'PTENGAXTA','PTMATAXTA',
#                                       'PTSCIAXTA']]


# ks2_2012 = ks2_2012.dropna(subset=["PTENGAXTA"])
# #change subjs to float
# #change la estab to int
# ks2_2012['LEA'] = ks2_2012['LEA'].astype(int)
# ks2_2012['ESTAB'] = ks2_2012['ESTAB'].astype(int)
# ks2_2012['LAESTAB'] = ks2_2012['LEA'].astype(str) + ks2_2012['ESTAB'].astype(str)
# ks2_2012["PTENGAXTA"] = ks2_2012["PTENGAXTA"].str.replace("%", "").astype(int)
# ks2_2012["PTMATAXTA"] = ks2_2012["PTMATAXTA"].str.replace("%", "").astype(int)
# ks2_2012["PTSCIAXTA"] = ks2_2012["PTSCIAXTA"].str.replace("%", "").astype(int)

# ks2_2012.rename(columns={"PTENGAXTA":"level5_english_p_12"},inplace=True)
# ks2_2012.rename(columns={"PTMATAXTA":"level5_maths_p_12"},inplace=True)
# ks2_2012.rename(columns={"PTSCIAXTA":"level5_science_p_12"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2012['english_rank'] = ks2_2012['level5_english_p_12'].rank(ascending=False)
# ks2_2012['maths_rank'] = ks2_2012['level5_maths_p_12'].rank(ascending=False)
# ks2_2012['science_rank'] = ks2_2012['level5_science_p_12'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2012['acad_rank'] = (ks2_2012['english_rank'] * 1/3 +
#               ks2_2012['maths_rank'] * 1/3 +
#               ks2_2012['science_rank'] * 1/3)

# #merge ks2 acad results with ofsted rating
# ks2_2012 = pd.merge(ks2_2012, ofsr_wide[['SCHNAME',2012]], how='left', on='SCHNAME')
# ks2_2012.rename(columns={2012:"ofsr"}, inplace=True)

# #manually input 
# ks2_2012.to_csv('schools/brighton/2012-2019/2011-2012_brighton_ks2_missing.csv', index=False)


ks2_2012 = pd.read_csv('schools/brighton/2012-2019/2011-2012_brighton_ks2_completed.csv')
#From 2012, the meaning of grade 3 changed from satisfactory to requires improvement.
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks2_2012['ofsr_values'] = ks2_2012['ofsr'].map(mapping)

fsm_2012 = pd.read_csv('schools/brighton/fsm/2011-2012_846_cfr.csv')
fsm_2012['URN'] = pd.to_numeric(fsm_2012['URN'], errors='coerce').dropna().astype(int)
ks2_2012 = pd.merge(ks2_2012, fsm_2012[['URN','FSM']], how='left', on='URN')
#Balfour Pri School is NA here, but we use 2011 data to replace it (3.9%)
ks2_2012['FSM'] = ks2_2012['FSM'].fillna(3.9)

#using ['acad_rank'], ['OFSR'] and ['FSM'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2012['rank'] = (ks2_2012['acad_rank'].rank(ascending=True) * 0.45 +
                    ks2_2012['ofsr_values'].rank(ascending=True, method='dense') * 0.45 +
                    ks2_2012['FSM'].rank(ascending=True) * 0.1
                   )
#we can then focus on the different types of schools for each year in the regression
ks2_2012.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2012['rank_band'] = pd.cut(ks2_2012['rank'], bins=bins, labels=labels)
#ks2_2012

In [16]:
## 44 pri schools in 2012. 44 schools in 2003-5, 07,09. (43 in 06, 08,11)
#building dataframe
ks2_2012_add = ks2_2012[['LAESTAB','SCHNAME', 'PCODE', 'rank_band']]
ks2_2012_add.rename(columns={'rank_band':"2012"}, inplace=True)
#ks2_2011_add["LAESTAB"] = ks2_2011_add["LAESTAB"].astype(int)
#left join to ks2_hist_df
ks2_hist_df = pd.read_csv('schools/brighton/ks2_hist_df.csv')
#Balfour Junior renamed to Balfour Pri, 8462000 to 8462040
ks2_hist_df["LAESTAB"] = ks2_hist_df["LAESTAB"].replace(8462000, 8462040)
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2012_add[['LAESTAB','2012']], how='left', on='LAESTAB')
ks2_hist_df.head()
ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)

In [11]:
# ## #2013
# #read in csv
# brighton_ks2_13 = pd.read_csv("schools/brighton/2012-2019/2012-2013_846_ks2final.csv")

# #drop non-school rows
# brighton_ks2_13.drop(brighton_ks2_13[(brighton_ks2_13['SCHNAME'] == 'Brighton and Hove')].index, inplace=True)
# brighton_ks2_13.drop(brighton_ks2_13[(brighton_ks2_13['SCHNAME'] == 'NATIONAL')].index, inplace=True)
# brighton_ks2_13.drop(brighton_ks2_13[(brighton_ks2_13['SCHNAME'] == 'NATIONALMTD')].index, inplace=True)
# brighton_ks2_13.drop(brighton_ks2_13[(brighton_ks2_13['NFTYPE'] == 'CYS')].index, inplace=True)

# #note Bilingual Pri School which has just started, but no data, its ofsted starts from 2014
# #drop Bilingual and City Acad Whitehawk, both no data 
# #extract key columns
# ks2_2013 = brighton_ks2_13[['LEA','ESTAB','URN', 'SCHNAME','PCODE',
#                                       'PTENGAXTA','PTMATAXTA',
#                                       'PTSCIAXTA']]


# ks2_2013 = ks2_2013.dropna(subset=["PTENGAXTA"])
# #change subjs to float
# #change la estab to int
# ks2_2013['LEA'] = ks2_2013['LEA'].astype(int)
# ks2_2013['ESTAB'] = ks2_2013['ESTAB'].astype(int)
# ks2_2013['LAESTAB'] = ks2_2013['LEA'].astype(str) + ks2_2013['ESTAB'].astype(str)
# ks2_2013["PTENGAXTA"] = ks2_2013["PTENGAXTA"].str.replace("%", "").astype(int)
# ks2_2013["PTMATAXTA"] = ks2_2013["PTMATAXTA"].str.replace("%", "").astype(int)
# ks2_2013["PTSCIAXTA"] = ks2_2013["PTSCIAXTA"].str.replace("%", "").astype(int)

# ks2_2013.rename(columns={"PTENGAXTA":"level5_english_p_13"},inplace=True)
# ks2_2013.rename(columns={"PTMATAXTA":"level5_maths_p_13"},inplace=True)
# ks2_2013.rename(columns={"PTSCIAXTA":"level5_science_p_13"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2013['english_rank'] = ks2_2013['level5_english_p_13'].rank(ascending=False)
# ks2_2013['maths_rank'] = ks2_2013['level5_maths_p_13'].rank(ascending=False)
# ks2_2013['science_rank'] = ks2_2013['level5_science_p_13'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2013['acad_rank'] = (ks2_2013['english_rank'] * 1/3 +
#               ks2_2013['maths_rank'] * 1/3 +
#               ks2_2013['science_rank'] * 1/3)

# #merge ks2 acad results with ofsted rating
# ks2_2013 = pd.merge(ks2_2013, ofsr_wide[['SCHNAME',2013]], how='left', on='SCHNAME')
# ks2_2013.rename(columns={2013:"ofsr"}, inplace=True)

# # #manually input 
# ks2_2013.to_csv('schools/brighton/2012-2019/2012-2013_brighton_ks2_missing.csv', index=False)

##step 2
ks2_2013 = pd.read_csv('schools/brighton/2012-2019/2012-2013_brighton_ks2_completed.csv')
#From 2012, the meaning of grade 3 changed from satisfactory to requires improvement.
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks2_2013['ofsr_values'] = ks2_2013['ofsr'].map(mapping)

fsm_2013 = pd.read_csv('schools/brighton/fsm/2012-2013_846_cfr.csv')
fsm_2013['URN'] = pd.to_numeric(fsm_2013['URN'], errors='coerce').dropna().astype(int)
ks2_2013 = pd.merge(ks2_2013, fsm_2013[['URN','FSM']], how='left', on='URN')

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2013['rank'] = (ks2_2013['acad_rank'].rank(ascending=True) * 0.45 +
                    ks2_2013['ofsr_values'].rank(ascending=True, method='dense') * 0.45 +
                    ks2_2013['FSM'].rank(ascending=True) *0.1
                   )
#we can then focus on the different types of schools for each year in the regression
ks2_2013.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2013['rank_band'] = pd.cut(ks2_2013['rank'], bins=bins, labels=labels)

In [12]:
# ## 44 pri schools in 2013. 44 schools in 2003-5, 07,09,12. (43 in 06, 08,11)
#building dataframe
ks2_2013_add = ks2_2013[['LAESTAB','SCHNAME', 'PCODE', 'rank_band']]
ks2_2013_add.rename(columns={'rank_band':"2013"}, inplace=True)
#ks2_2011_add["LAESTAB"] = ks2_2011_add["LAESTAB"].astype(int)
#left join to ks2_hist_df
ks2_hist_df = pd.read_csv('schools/brighton/ks2_hist_df.csv')
#Balfour Junior renamed to Balfour Pri, 8462000 to 8462040
#ks2_hist_df["LAESTAB"] = ks2_hist_df["LAESTAB"].replace(8462000, 8462040)
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2013_add[['LAESTAB','2013']], how='left', on='LAESTAB')
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)

In [15]:
ks2_2013_add

,LAESTAB,SCHNAME,PCODE,2013
25,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding
9,8462009,Downs Junior School,BN1 6ED,Outstanding
41,8462036,Westdene Primary School,BN1 5GN,Outstanding
23,8463329,St Andrew's CofE (Aided) Primary School,BN3 3YT,Outstanding
38,8462028,Stanford Junior School,BN1 5PR,Outstanding
40,8462070,West Hove Junior School,BN3 5JA,Outstanding
1,8462040,Balfour Primary School,BN1 6NE,Outstanding
28,8462026,St Luke's Primary School,BN2 9ZF,Outstanding
18,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding
8,8463341,Cottesmore St Mary's Catholic Primary School,BN3 6NB,Outstanding


In [22]:
## 2014

# #read in csv
# brighton_ks2_14 = pd.read_csv("schools/brighton/2012-2019/2013-2014_846_ks2final.csv")

# #drop non-school rows
# brighton_ks2_14.drop(brighton_ks2_14[(brighton_ks2_14['SCHNAME'] == 'Brighton and Hove')].index, inplace=True)
# brighton_ks2_14.drop(brighton_ks2_14[(brighton_ks2_14['SCHNAME'] == 'NATIONAL')].index, inplace=True)
# brighton_ks2_14.drop(brighton_ks2_14[(brighton_ks2_14['SCHNAME'] == 'NATIONALMTD')].index, inplace=True)
# brighton_ks2_14.drop(brighton_ks2_14[(brighton_ks2_14['NFTYPE'] == 'CYS')].index, inplace=True)

# #note Bilingual Pri School which has just started, but no data, its ofsted starts from 2014
# #drop Bilingual and City Acad Whitehawk, both no data yet
# #Brackenbury opened Sept 2013, appears here but no data
# #extract key columns
# ks2_2014 = brighton_ks2_14[['LEA','ESTAB','URN', 'SCHNAME','PCODE',
#                                       'PTENGAXTA','PTMATAXTA',
#                                       'PTSCIAXTA']]


# ks2_2014 = ks2_2014.dropna(subset=["PTENGAXTA"])
# #change subjs to float
# #change la estab to int
# ks2_2014['LEA'] = ks2_2014['LEA'].astype(int)
# ks2_2014['ESTAB'] = ks2_2014['ESTAB'].astype(int)
# ks2_2014['LAESTAB'] = ks2_2014['LEA'].astype(str) + ks2_2014['ESTAB'].astype(str)
# ks2_2014["PTENGAXTA"] = ks2_2014["PTENGAXTA"].str.replace("%", "").astype(int)
# ks2_2014["PTMATAXTA"] = ks2_2014["PTMATAXTA"].str.replace("%", "").astype(int)
# ks2_2014["PTSCIAXTA"] = ks2_2014["PTSCIAXTA"].str.replace("%", "").astype(int)

# ks2_2014.rename(columns={"PTENGAXTA":"level5_english_p_14"},inplace=True)
# ks2_2014.rename(columns={"PTMATAXTA":"level5_maths_p_14"},inplace=True)
# ks2_2014.rename(columns={"PTSCIAXTA":"level5_science_p_14"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2014['english_rank'] = ks2_2014['level5_english_p_14'].rank(ascending=False)
# ks2_2014['maths_rank'] = ks2_2014['level5_maths_p_14'].rank(ascending=False)
# ks2_2014['science_rank'] = ks2_2014['level5_science_p_14'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2014['acad_rank'] = (ks2_2014['english_rank'] * 1/3 +
#               ks2_2014['maths_rank'] * 1/3 +
#               ks2_2014['science_rank'] * 1/3)

# #merge ks2 acad results with ofsted rating
# ks2_2014 = pd.merge(ks2_2014, ofsr_wide[['SCHNAME',2014]], how='left', on='SCHNAME')
# ks2_2014.rename(columns={2014:"ofsr"}, inplace=True)

# #manually input 
# ks2_2014.to_csv('schools/brighton/2012-2019/2013-2014_brighton_ks2_missing.csv', index=False)

##step 2
ks2_2014 = pd.read_csv('schools/brighton/2012-2019/2013-2014_brighton_ks2_completed.csv')
#From 2012, the meaning of grade 3 changed from satisfactory to requires improvement.
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks2_2014['ofsr_values'] = ks2_2014['ofsr'].map(mapping)

fsm_2014 = pd.read_csv('schools/brighton/fsm/2013-2014_846_cfr.csv')
fsm_2014['URN'] = pd.to_numeric(fsm_2014['URN'], errors='coerce').dropna().astype(int)
ks2_2014 = pd.merge(ks2_2014, fsm_2014[['URN','FSM']], how='left', on='URN')

#Whitehawk Pri School is NA here, but we use 2013 data to replace it (3.9%)
ks2_2014['FSM'] = ks2_2014['FSM'].fillna(51.9)

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2014['rank'] = (ks2_2014['acad_rank'].rank(ascending=True) * 0.45 +
                    ks2_2014['ofsr_values'].rank(ascending=True, method='dense') * 0.45 +
                    ks2_2014['FSM'].rank(ascending=True) *0.1
                   )
#we can then focus on the different types of schools for each year in the regression
ks2_2014.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2014['rank_band'] = pd.cut(ks2_2014['rank'], bins=bins, labels=labels)

In [24]:
## 44 pri schools in 2014. 44 schools in 2003-5, 07,09,12-13. (43 in 06, 08,11)
#building dataframe
ks2_2014_add = ks2_2014[['LAESTAB','SCHNAME', 'PCODE', 'rank_band']]
ks2_2014_add.rename(columns={'rank_band':"2014"}, inplace=True)
#ks2_2011_add["LAESTAB"] = ks2_2011_add["LAESTAB"].astype(int)
#left join to ks2_hist_df
ks2_hist_df = pd.read_csv('schools/brighton/ks2_hist_df.csv')
#Whitehawk NaN
#Whitehawk Primary School to City Academy Whitehawk (SCHNAME)
#8462006to 8462011
# ks2_hist_df["LAESTAB"] = ks2_hist_df["LAESTAB"].replace(8462006, 8462011)
# ks2_hist_df["School name"] = ks2_hist_df["School name"].replace('Whitehawk Primary School', 'City Academy Whitehawk')
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2014_add[['LAESTAB','2014']], how='left', on='LAESTAB')
ks2_hist_df.head()
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)

In [31]:
# # 2015
# #read in csv
# brighton_ks2_15 = pd.read_csv("schools/brighton/2012-2019/2014-2015_846_ks2final.csv")

# #drop non-school rows
# brighton_ks2_15.drop(brighton_ks2_15[(brighton_ks2_15['SCHNAME'] == 'Brighton and Hove')].index, inplace=True)
# brighton_ks2_15.drop(brighton_ks2_15[(brighton_ks2_15['SCHNAME'] == 'NATIONAL')].index, inplace=True)
# brighton_ks2_15.drop(brighton_ks2_15[(brighton_ks2_15['SCHNAME'] == 'NATIONALMTD')].index, inplace=True)
# brighton_ks2_15.drop(brighton_ks2_15[(brighton_ks2_15['NFTYPE'] == 'CYS')].index, inplace=True)

# #note Bilingual Pri School which has just started, but no data, its ofsted starts from 2014
# #Brackenbury opened Sept 2013, appears here but no data
# # St Peter's also new, but no ks2 data yet (used to be 3-7 age range, infant and nursery sch)
# #Drop Bilingual, Brackenbury and St Peters for 2015
# #extract key columns
# ks2_2015 = brighton_ks2_15[['LEA','ESTAB','URN', 'SCHNAME','PCODE',
#                                       'PTENGAXTA','PTMATAXTA',
#                                       'PTSCIAXTA']]


# ks2_2015 = ks2_2015.dropna(subset=["PTENGAXTA"])
# #change subjs to float
# #change la estab to int
# ks2_2015['LEA'] = ks2_2015['LEA'].astype(int)
# ks2_2015['ESTAB'] = ks2_2015['ESTAB'].astype(int)
# ks2_2015['LAESTAB'] = ks2_2015['LEA'].astype(str) + ks2_2015['ESTAB'].astype(str)
# ks2_2015["PTENGAXTA"] = ks2_2015["PTENGAXTA"].str.replace("%", "").astype(int)
# ks2_2015["PTMATAXTA"] = ks2_2015["PTMATAXTA"].str.replace("%", "").astype(int)
# ks2_2015["PTSCIAXTA"] = ks2_2015["PTSCIAXTA"].str.replace("%", "").astype(int)

# ks2_2015.rename(columns={"PTENGAXTA":"level5_english_p_15"},inplace=True)
# ks2_2015.rename(columns={"PTMATAXTA":"level5_maths_p_15"},inplace=True)
# ks2_2015.rename(columns={"PTSCIAXTA":"level5_science_p_15"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2015['english_rank'] = ks2_2015['level5_english_p_15'].rank(ascending=False)
# ks2_2015['maths_rank'] = ks2_2015['level5_maths_p_15'].rank(ascending=False)
# ks2_2015['science_rank'] = ks2_2015['level5_science_p_15'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2015['acad_rank'] = (ks2_2015['english_rank'] * 1/3 +
#               ks2_2015['maths_rank'] * 1/3 +
#               ks2_2015['science_rank'] * 1/3)

# #merge ks2 acad results with ofsted rating
# ks2_2015 = pd.merge(ks2_2015, ofsr_wide[['SCHNAME',2015]], how='left', on='SCHNAME')
# ks2_2015.rename(columns={2015:"ofsr"}, inplace=True)

# #manually input 
# ks2_2015.to_csv('schools/brighton/2012-2019/2014-2015_brighton_ks2_missing.csv', index=False)

#step 2
ks2_2015 = pd.read_csv('schools/brighton/2012-2019/2014-2015_brighton_ks2_completed.csv')
#From 2012, the meaning of grade 3 changed from satisfactory to requires improvement.
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks2_2015['ofsr_values'] = ks2_2015['ofsr'].map(mapping)

fsm_2015 = pd.read_csv('schools/brighton/fsm/2014-2015_846_cfr.csv')
fsm_2015['URN'] = pd.to_numeric(fsm_2015['URN'], errors='coerce').dropna().astype(int)
ks2_2015 = pd.merge(ks2_2015, fsm_2015[['URN','FSM']], how='left', on='URN')

#whitehawk nan again
ks2_2015['FSM'] = ks2_2015['FSM'].fillna(51.9)

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2015['rank'] = (ks2_2015['acad_rank'].rank(ascending=True) * 0.45 +
                    ks2_2015['ofsr_values'].rank(ascending=True, method='dense') * 0.45 +
                    ks2_2015['FSM'].rank(ascending=True) *0.1
                   )
#we can then focus on the different types of schools for each year in the regression
ks2_2015.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2015['rank_band'] = pd.cut(ks2_2015['rank'], bins=bins, labels=labels)

In [34]:
## 44 pri schools in 2015. 44 schools in 2003-5, 07,09,12-14. (43 in 06, 08,11)
#building dataframe
ks2_2015_add = ks2_2015[['LAESTAB','SCHNAME', 'PCODE', 'rank_band']]
ks2_2015_add.rename(columns={'rank_band':"2015"}, inplace=True)
#ks2_2011_add["LAESTAB"] = ks2_2011_add["LAESTAB"].astype(int)
#left join to ks2_hist_df
ks2_hist_df = pd.read_csv('schools/brighton/ks2_hist_df.csv')

ks2_hist_df = pd.merge(ks2_hist_df, ks2_2015_add[['LAESTAB','2015']], how='left', on='LAESTAB')
ks2_hist_df.head()
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)

,LAESTAB,School name,School postcode,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2014,2015
0,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
1,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding,Outstanding,Outstanding,Outstanding,Good,Good,Good,Outstanding,Good,Outstanding,Good,Good
2,8462040,Balfour Junior School,BN1 6NE,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
3,8462028,Stanford Junior School,BN1 5PR,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
4,8462036,Westdene Primary School,BN1 5GN,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding


In [44]:
## # 2016
# #read in csv
# brighton_ks2_16 = pd.read_csv("schools/brighton/2012-2019/2015-2016_846_ks2final.csv")

# #drop non-school rows
# # brighton_ks2_16.drop(brighton_ks2_16[(brighton_ks2_16['SCHNAME'] == 'Brighton and Hove')].index, inplace=True)
# # brighton_ks2_16.dropna(subset=['SCHNAME'], inplace=True)
# # brighton_ks2_16.drop(brighton_ks2_16[(brighton_ks2_16['NFTYPE'] == 'CYS')].index, inplace=True)
# brighton_ks2_16 = brighton_ks2_16[(brighton_ks2_16['RECTYPE'] == 1)]

# #note Bilingual Pri School which has just started, but no data, its ofsted starts from 2014
# #Brackenbury opened Sept 2013, appears here but no data
# #St Peter's also new, but no ks2 data yet (used to be 3-7 age range, infant and nursery sch)
# #Drop Bilingual, Brackenbury and St Peters for 2016
# #extract key columns
# ks2_2016 = brighton_ks2_16[['LEA','ESTAB','URN', 'SCHNAME','PCODE',
#                                       'PTRWM_EXP', 'PTSCITA_EXP']]
# ks2_2016 = ks2_2016.dropna(subset=["PTRWM_EXP"])

# #change subjs to float
# #change la estab to int
# ks2_2016['LEA'] = ks2_2016['LEA'].astype(int)
# ks2_2016['ESTAB'] = ks2_2016['ESTAB'].astype(int)
# ks2_2016['LAESTAB'] = ks2_2016['LEA'].astype(str) + ks2_2016['ESTAB'].astype(str)
# ks2_2016["PTRWM_EXP"] = ks2_2016["PTRWM_EXP"].str.replace("%", "").astype(int)
# ks2_2016["PTSCITA_EXP"] = ks2_2016["PTSCITA_EXP"].str.replace("%", "").astype(int)

# ks2_2016.rename(columns={"PTRWM_EXP":"exp_wrm_p_16"},inplace=True)
# ks2_2016.rename(columns={"PTSCITA_EXP":"exp_sci_p_16"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2016['wrm_rank'] = ks2_2016['exp_wrm_p_16'].rank(ascending=False)
# ks2_2016['science_rank'] = ks2_2016['exp_sci_p_16'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2016['acad_rank'] = (ks2_2016['wrm_rank'] * 0.75 +
#               ks2_2016['science_rank'] * 0.25)

# #merge ks2 acad results with ofsted rating
# ks2_2016 = pd.merge(ks2_2016, ofsr_wide[['SCHNAME',2016]], how='left', on='SCHNAME')
# ks2_2016.rename(columns={2016:"ofsr"}, inplace=True)

# #manually input 
# ks2_2016.to_csv('schools/brighton/2012-2019/2015-2016_brighton_ks2_missing.csv', index=False)

#step 2
ks2_2016 = pd.read_csv('schools/brighton/2012-2019/2015-2016_brighton_ks2_completed.csv')
#From 2012, the meaning of grade 3 changed from satisfactory to requires improvement.
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks2_2016['ofsr_values'] = ks2_2016['ofsr'].map(mapping)

fsm_2016 = pd.read_csv('schools/brighton/fsm/2015-2016_846_cfr.csv')
fsm_2016['URN'] = pd.to_numeric(fsm_2016['URN'], errors='coerce').dropna().astype(int)
ks2_2016 = pd.merge(ks2_2016, fsm_2016[['URN','FSM']], how='left', on='URN')

#whitehawk nan again
ks2_2016['FSM'] = ks2_2016['FSM'].fillna(51.9)

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2016['rank'] = (ks2_2016['acad_rank'].rank(ascending=True) * 0.45 +
                    ks2_2016['ofsr_values'].rank(ascending=True, method='dense') * 0.45 +
                    ks2_2016['FSM'].rank(ascending=True) *0.1
                   )
#we can then focus on the different types of schools for each year in the regression
ks2_2016.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2016['rank_band'] = pd.cut(ks2_2016['rank'], bins=bins, labels=labels)

In [51]:
## 44 pri schools in 2016. 44 schools in 2003-5, 07,09,12-15. (43 in 06, 08,11)
#building dataframe
ks2_2016_add = ks2_2016[['LAESTAB','SCHNAME', 'PCODE', 'rank_band']]
ks2_2016_add.rename(columns={'rank_band':"2016"}, inplace=True)
#ks2_2011_add["LAESTAB"] = ks2_2011_add["LAESTAB"].astype(int)
#left join to ks2_hist_df
ks2_hist_df = pd.read_csv('schools/brighton/ks2_hist_df.csv')
#Hangleton NaN
#Hangleton Junior School to Hangleton Primary School (SCHNAME)
#8462067to 8462066
ks2_hist_df["LAESTAB"] = ks2_hist_df["LAESTAB"].replace(8462067, 8462066)
ks2_hist_df["School name"] = ks2_hist_df["School name"].replace('Hangleton Junior School', 'Hangleton Primary School')
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2016_add[['LAESTAB','2016']], how='left', on='LAESTAB')
ks2_hist_df.head()
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)

,LAESTAB,School name,School postcode,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2014,2015,2016
0,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
1,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding,Outstanding,Outstanding,Outstanding,Good,Good,Good,Outstanding,Good,Outstanding,Good,Good,Good
2,8462040,Balfour Junior School,BN1 6NE,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Good
3,8462028,Stanford Junior School,BN1 5PR,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
4,8462036,Westdene Primary School,BN1 5GN,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding


In [59]:
#2017
# #read in csv
# brighton_ks2_17 = pd.read_csv("schools/brighton/2012-2019/2016-2017_846_ks2final.csv")

# #drop non-school rows
# # brighton_ks2_16.drop(brighton_ks2_16[(brighton_ks2_16['SCHNAME'] == 'Brighton and Hove')].index, inplace=True)
# # brighton_ks2_16.dropna(subset=['SCHNAME'], inplace=True)
# # brighton_ks2_16.drop(brighton_ks2_16[(brighton_ks2_16['NFTYPE'] == 'CYS')].index, inplace=True)
# brighton_ks2_17 = brighton_ks2_17[(brighton_ks2_17['RECTYPE'] == 1)]

# #note Bilingual Pri School which has just started, but no data, its ofsted starts from 2014
# #Brackenbury opened Sept 2013, appears here but no data
# #St Peter's also new, but no ks2 data yet (used to be 3-7 age range, infant and nursery sch)
# #Drop Bilingual, Brackenbury and St Peters for 2017
# #extract key columns
# ks2_2017 = brighton_ks2_17[['LEA','ESTAB','URN', 'SCHNAME','PCODE',
#                                       'PTRWM_EXP', 'PTSCITA_EXP']]
# ks2_2017 = ks2_2017.dropna(subset=["PTRWM_EXP"])

# #change subjs to float
# #change la estab to int
# ks2_2017['LEA'] = ks2_2017['LEA'].astype(int)
# ks2_2017['ESTAB'] = ks2_2017['ESTAB'].astype(int)
# ks2_2017['LAESTAB'] = ks2_2017['LEA'].astype(str) + ks2_2017['ESTAB'].astype(str)
# ks2_2017["PTRWM_EXP"] = ks2_2017["PTRWM_EXP"].str.replace("%", "").astype(int)
# ks2_2017["PTSCITA_EXP"] = ks2_2017["PTSCITA_EXP"].str.replace("%", "").astype(int)

# ks2_2017.rename(columns={"PTRWM_EXP":"exp_wrm_p_17"},inplace=True)
# ks2_2017.rename(columns={"PTSCITA_EXP":"exp_sci_p_17"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2017['wrm_rank'] = ks2_2017['exp_wrm_p_17'].rank(ascending=False)
# ks2_2017['science_rank'] = ks2_2017['exp_sci_p_17'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2017['acad_rank'] = (ks2_2017['wrm_rank'] * 0.75 +
#               ks2_2017['science_rank'] * 0.25)

# #merge ks2 acad results with ofsted rating
# ks2_2017 = pd.merge(ks2_2017, ofsr_wide[['SCHNAME',2017]], how='left', on='SCHNAME')
# ks2_2017.rename(columns={2017:"ofsr"}, inplace=True)

# #manually input 
# ks2_2017.to_csv('schools/brighton/2012-2019/2016-2017_brighton_ks2_missing.csv', index=False)

#step 2
ks2_2017 = pd.read_csv('schools/brighton/2012-2019/2016-2017_brighton_ks2_completed.csv')
#From 2012, the meaning of grade 3 changed from satisfactory to requires improvement.
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks2_2017['ofsr_values'] = ks2_2017['ofsr'].map(mapping)

fsm_2017 = pd.read_csv('schools/brighton/fsm/2016-2017_846_cfr.csv')
fsm_2017['URN'] = pd.to_numeric(fsm_2017['URN'], errors='coerce').dropna().astype(int)
ks2_2017 = pd.merge(ks2_2017, fsm_2017[['URN','FSM']], how='left', on='URN')

#whitehawk nan again
ks2_2017['FSM'] = ks2_2017['FSM'].fillna(51.9)

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2017['rank'] = (ks2_2017['acad_rank'].rank(ascending=True) * 0.45 +
                    ks2_2017['ofsr_values'].rank(ascending=True, method='dense') * 0.45 +
                    ks2_2017['FSM'].rank(ascending=True) *0.1
                   )
#we can then focus on the different types of schools for each year in the regression
ks2_2017.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2017['rank_band'] = pd.cut(ks2_2017['rank'], bins=bins, labels=labels)

In [60]:
## 44 pri schools in 2017. 44 schools in 2003-5, 07,09,12-16. (43 in 06, 08,11)
#building dataframe
ks2_2017_add = ks2_2017[['LAESTAB','SCHNAME', 'PCODE', 'rank_band']]
ks2_2017_add.rename(columns={'rank_band':"2017"}, inplace=True)
#ks2_2011_add["LAESTAB"] = ks2_2011_add["LAESTAB"].astype(int)
#left join to ks2_hist_df
ks2_hist_df = pd.read_csv('schools/brighton/ks2_hist_df.csv')
#Hangleton NaN
#Hangleton Junior School to Hangleton Primary School (SCHNAME)
#8462067to 8462066
# ks2_hist_df["LAESTAB"] = ks2_hist_df["LAESTAB"].replace(8462067, 8462066)
# ks2_hist_df["School name"] = ks2_hist_df["School name"].replace('Hangleton Junior School', 'Hangleton Primary School')
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2017_add[['LAESTAB','2017']], how='left', on='LAESTAB')
ks2_hist_df.head()
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)

,LAESTAB,School name,School postcode,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2014,2015,2016,2017
0,8463318,St Bernadette's Catholic Primary School,BN1 6UT,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
1,8463316,Our Lady of Lourdes RC School,BN2 7HA,Outstanding,Outstanding,Outstanding,Outstanding,Good,Good,Good,Outstanding,Good,Outstanding,Good,Good,Good,Satisfactory
2,8462040,Balfour Junior School,BN1 6NE,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Good,Good
3,8462028,Stanford Junior School,BN1 5PR,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding
4,8462036,Westdene Primary School,BN1 5GN,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding,Outstanding


In [79]:
#2018
# #read in csv
# brighton_ks2_18 = pd.read_csv("schools/brighton/2012-2019/2017-2018_846_ks2final.csv")

# #drop non-school rows
# # brighton_ks2_16.drop(brighton_ks2_16[(brighton_ks2_16['SCHNAME'] == 'Brighton and Hove')].index, inplace=True)
# # brighton_ks2_16.dropna(subset=['SCHNAME'], inplace=True)
# # brighton_ks2_16.drop(brighton_ks2_16[(brighton_ks2_16['NFTYPE'] == 'CYS')].index, inplace=True)
# brighton_ks2_18 = brighton_ks2_18[(brighton_ks2_18['RECTYPE'] == 1)]


# #MAKE SURE Bilingual, Brackenbury and St Peters are in for 2018
# #extract key columns
# ks2_2018 = brighton_ks2_18[['LEA','ESTAB','URN', 'SCHNAME','PCODE',
#                                       'PTRWM_EXP', 'PTSCITA_EXP']]
# ks2_2018 = ks2_2018.dropna(subset=["PTRWM_EXP"])

# #change subjs to float
# #change la estab to int
# ks2_2018['LEA'] = ks2_2018['LEA'].astype(int)
# ks2_2018['ESTAB'] = ks2_2018['ESTAB'].astype(int)
# ks2_2018['LAESTAB'] = ks2_2018['LEA'].astype(str) + ks2_2018['ESTAB'].astype(str)
# ks2_2018["PTRWM_EXP"] = ks2_2018["PTRWM_EXP"].str.replace("%", "").astype(int)
# ks2_2018["PTSCITA_EXP"] = ks2_2018["PTSCITA_EXP"].str.replace("%", "").astype(int)

# ks2_2018.rename(columns={"PTRWM_EXP":"exp_wrm_p_18"},inplace=True)
# ks2_2018.rename(columns={"PTSCITA_EXP":"exp_sci_p_18"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2018['wrm_rank'] = ks2_2018['exp_wrm_p_18'].rank(ascending=False)
# ks2_2018['science_rank'] = ks2_2018['exp_sci_p_18'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2018['acad_rank'] = (ks2_2018['wrm_rank'] * 0.75 +
#               ks2_2018['science_rank'] * 0.25)

# #merge ks2 acad results with ofsted rating
# ks2_2018 = pd.merge(ks2_2018, ofsr_wide[['SCHNAME',2018]], how='left', on='SCHNAME')
# ks2_2018.rename(columns={2018:"ofsr"}, inplace=True)

# #manually input 
# ks2_2018.to_csv('schools/brighton/2012-2019/2017-2018_brighton_ks2_missing.csv', index=False)

#step 2
ks2_2018 = pd.read_csv('schools/brighton/2012-2019/2017-2018_brighton_ks2_completed.csv')
#From 2012, the meaning of grade 3 changed from satisfactory to requires improvement.
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks2_2018['ofsr_values'] = ks2_2018['ofsr'].map(mapping)

fsm_2018 = pd.read_csv('schools/brighton/fsm/2017-2018_846_cfr.csv')
fsm_2018['URN'] = pd.to_numeric(fsm_2018['URN'], errors='coerce').dropna().astype(int)
ks2_2018 = pd.merge(ks2_2018, fsm_2018[['URN','FSM']], how='left', on='URN')

# #whitehawk nan again, bilingual nan too
#replace whitehawk with 65.9 (2023 value)
ks2_2018.loc[ks2_2018['URN'] == 139677, 'FSM'] = 65.9
#replace bilingual with 13.7 (2023 value)
ks2_2018.loc[ks2_2018['URN'] == 138261, 'FSM'] = 13.7

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2018['rank'] = (ks2_2018['acad_rank'].rank(ascending=True) * 0.45 +
                    ks2_2018['ofsr_values'].rank(ascending=True, method='dense') * 0.45 +
                    ks2_2018['FSM'].rank(ascending=True) *0.1
                   )
#we can then focus on the different types of schools for each year in the regression
ks2_2018.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2018['rank_band'] = pd.cut(ks2_2018['rank'], bins=bins, labels=labels)

In [82]:
## 47 pri schools in 2018. 44 schools in 2003-5, 07,09,12-17. (43 in 06, 08,11)
#building dataframe
ks2_2018_add = ks2_2018[['LAESTAB','SCHNAME', 'PCODE', 'rank_band']]
ks2_2018_add.rename(columns={'rank_band':"2018"}, inplace=True)
#ks2_2011_add["LAESTAB"] = ks2_2011_add["LAESTAB"].astype(int)
#left join to ks2_hist_df
ks2_hist_df = pd.read_csv('schools/brighton/ks2_hist_df.csv')
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2018_add[['LAESTAB','2018']], how='outer', on='LAESTAB')
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)
#add schoolname and postocde to missing schools 

In [85]:
# 2019
# #read in csv
# brighton_ks2_19 = pd.read_csv("schools/brighton/2012-2019/2018-2019_846_ks2final.csv")

# #drop non-school rows
# # brighton_ks2_16.drop(brighton_ks2_16[(brighton_ks2_16['SCHNAME'] == 'Brighton and Hove')].index, inplace=True)
# # brighton_ks2_16.dropna(subset=['SCHNAME'], inplace=True)
# # brighton_ks2_16.drop(brighton_ks2_16[(brighton_ks2_16['NFTYPE'] == 'CYS')].index, inplace=True)
# brighton_ks2_19 = brighton_ks2_19[(brighton_ks2_19['RECTYPE'] == 1)]


# #MAKE SURE Bilingual, Brackenbury and St Peters are in for 2018
# #extract key columns
# ks2_2019 = brighton_ks2_19[['LEA','ESTAB','URN', 'SCHNAME','PCODE',
#                                       'PTRWM_EXP', 'PTSCITA_EXP']]
# ks2_2019 = ks2_2019.dropna(subset=["PTRWM_EXP"])

# #change subjs to float
# #change la estab to int
# ks2_2019['LEA'] = ks2_2019['LEA'].astype(int)
# ks2_2019['ESTAB'] = ks2_2019['ESTAB'].astype(int)
# ks2_2019['LAESTAB'] = ks2_2019['LEA'].astype(str) + ks2_2019['ESTAB'].astype(str)
# ks2_2019["PTRWM_EXP"] = ks2_2019["PTRWM_EXP"].str.replace("%", "").astype(int)
# ks2_2019["PTSCITA_EXP"] = ks2_2019["PTSCITA_EXP"].str.replace("%", "").astype(int)

# ks2_2019.rename(columns={"PTRWM_EXP":"exp_wrm_p_19"},inplace=True)
# ks2_2019.rename(columns={"PTSCITA_EXP":"exp_sci_p_19"},inplace=True)

# #using the percentage of students who reached level for each subject (eng, math, sci), we rank the schools
# ks2_2019['wrm_rank'] = ks2_2019['exp_wrm_p_19'].rank(ascending=False)
# ks2_2019['science_rank'] = ks2_2019['exp_sci_p_19'].rank(ascending=False)
# #we assign equal weights to each subject
# #weights = {'english_rank': 1/3, 'maths_rank': 1/3, 'science_rank': 1/3}
# #Calculate the weighted sum for each school by multiplying the rank of each variable with its corresponding weight and summing them up 
# #--> obtain academic ranking
# ks2_2019['acad_rank'] = (ks2_2019['wrm_rank'] * 0.75 +
#               ks2_2019['science_rank'] * 0.25)

# #merge ks2 acad results with ofsted rating
# ks2_2019 = pd.merge(ks2_2019, ofsr_wide[['SCHNAME',2019]], how='left', on='SCHNAME')
# ks2_2019.rename(columns={2019:"ofsr"}, inplace=True)

# #manually input 
# ks2_2019.to_csv('schools/brighton/2012-2019/2018-2019_brighton_ks2_missing.csv', index=False)

#step 2
ks2_2019 = pd.read_csv('schools/brighton/2012-2019/2018-2019_brighton_ks2_completed.csv')
#From 2012, the meaning of grade 3 changed from satisfactory to requires improvement.
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks2_2019['ofsr_values'] = ks2_2019['ofsr'].map(mapping)

fsm_2019 = pd.read_csv('schools/brighton/fsm/2018-2019_846_cfr.csv')
fsm_2019['URN'] = pd.to_numeric(fsm_2019['URN'], errors='coerce').dropna().astype(int)
ks2_2019 = pd.merge(ks2_2019, fsm_2019[['URN','FSM']], how='left', on='URN')

# #whitehawk nan again, bilingual nan too
#replace whitehawk with 65.9 (2023 value)
ks2_2019.loc[ks2_2019['URN'] == 139677, 'FSM'] = 65.9
#replace bilingual with 13.7 (2023 value)
ks2_2019.loc[ks2_2019['URN'] == 138261, 'FSM'] = 13.7

#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks2_2019['rank'] = (ks2_2019['acad_rank'].rank(ascending=True) * 0.45 +
                    ks2_2019['ofsr_values'].rank(ascending=True, method='dense') * 0.45 +
                    ks2_2019['FSM'].rank(ascending=True) *0.1
                   )
#we can then focus on the different types of schools for each year in the regression
ks2_2019.sort_values('rank', ascending=True, inplace=True)
bins = 4
labels = ['Outstanding', 'Good', 'Satisfactory', 'Poor']
ks2_2019['rank_band'] = pd.cut(ks2_2019['rank'], bins=bins, labels=labels)

In [87]:
## 47 pri schools in 2019. 44 schools in 2003-5, 07,09,12-18. (43 in 06, 08,11)
#building dataframe
ks2_2019_add = ks2_2019[['LAESTAB','SCHNAME', 'PCODE', 'rank_band']]
ks2_2019_add.rename(columns={'rank_band':"2019"}, inplace=True)
#ks2_2011_add["LAESTAB"] = ks2_2011_add["LAESTAB"].astype(int)
#left join to ks2_hist_df
ks2_hist_df = pd.read_csv('schools/brighton/ks2_hist_df.csv')
ks2_hist_df = pd.merge(ks2_hist_df, ks2_2019_add[['LAESTAB','2019']], how='left', on='LAESTAB')
#ks2_hist_df.to_csv('schools/brighton/ks2_hist_df.csv', index=False)

# Building up the school performance dataset: Secondary Schools 2000-2019

To build up the historical school performance dataset, we will build the dataset using the first 2003 dataset (but containing data from 2000 for secondary school gcses), and left join each subsequent year's rankings to the dataset. This will allow us to account for changes in primary schools (opening/ closing) for each year.

Starting with the 2003 dataset and repeated for each subsequent year

Read in england dataset for the year
Extract Brighton schools and key columns
Create new column LAESTAB that combines LEA/LA and ESTAB number (using regex)
Rename columns
Use variables to sort and rank schools
Drop any empty schools
Left join yearly rankings to main dataframe
Save csv
For ranking, 2003-2006 is GCSEs, 2007-2011 is GCSEs and OFSR, 2012-2019 is GCSEs, OFSR, pupil destination and FSMP

## 2000-2006 GCSEs

### 2000-2003

In [7]:
# 2000-2003
# #read in xls
# england_ks4_03 = pd.read_excel("schools/brighton/ks4/2002-2003_england_ks4final.xls", sheet_name=1, header=0)
# #extract brighton schools
# brighton_ks4_03 = england_ks4_03[england_ks4_03['LEA number']== 846]
# brighton_ks4_03 = brighton_ks4_03[brighton_ks4_03['record type']== 1]
# #extract key columns
# ks4_2003 = brighton_ks4_03[['LEA number','ESTAB number','institution name','institution postcode',
#                             '% of 15 year old pupils achieving 5 or more grades A*-C (GCSE/GNVQ) - 2000',
#                             '% of 15 year old pupils achieving 5 or more grades A*-C (GCSE/GNVQ) - 2001',
#                             '% of 15 year old pupils achieving 5 or more grades A*-C (GCSE/GNVQ) - 2002 ', 
#                             '% of 15 year old pupils achieving 5 or more grades A*-C (GCSE/GNVQ) - 2003 ']]
# ks4_2003['LAESTAB'] = ks4_2003['LEA number'].astype(str) + ks4_2003['ESTAB number'].astype(str)
# ks4_2003.rename(columns={"% of 15 year old pupils achieving 5 or more grades A*-C (GCSE/GNVQ) - 2000":"gcse_00"},inplace=True)
# ks4_2003.rename(columns={"% of 15 year old pupils achieving 5 or more grades A*-C (GCSE/GNVQ) - 2001":"gcse_01"},inplace=True)
# ks4_2003.rename(columns={"% of 15 year old pupils achieving 5 or more grades A*-C (GCSE/GNVQ) - 2002 ":"gcse_02"},inplace=True)
# ks4_2003.rename(columns={"% of 15 year old pupils achieving 5 or more grades A*-C (GCSE/GNVQ) - 2003 ":"gcse_03"},inplace=True)

# ks4_2003['gcse_00'] = pd.to_numeric(ks4_2003['gcse_00'], errors='coerce')
# ks4_2003['gcse_01'] = pd.to_numeric(ks4_2003['gcse_01'], errors='coerce')
# ks4_2003['gcse_02'] = pd.to_numeric(ks4_2003['gcse_02'], errors='coerce')
# ks4_2003['gcse_03'] = pd.to_numeric(ks4_2003['gcse_03'], errors='coerce')

# #save in case 
# ks4_2003.to_csv('schools/brighton/2002-2006/2000-2003_brighton_ks4check.csv', index=False)

# manually removed independent schools that are outside of research scope
# #BASED ON ADAM FEEDBACK
ks4_2003 = pd.read_csv('schools/brighton/2002-2006/2000-2003_brighton_ks4final.csv')
#year00 rank
ks4_2003 = ks4_2003.dropna(subset=["gcse_00"])
ks4_2003['00_rank'] = ks4_2003['gcse_00'].rank(ascending=False)
ks4_2003.sort_values('00_rank', ascending=True, inplace=True)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']
ks4_2003['00_rank_band'] = pd.cut(ks4_2003['00_rank'], bins=bins, labels=labels)

#building dataframe
ks4_2003.rename(columns={"institution name":"School name"},inplace=True)
ks4_2003.rename(columns={"institution postcode":"School postcode"},inplace=True)
ks4_hist_df = ks4_2003[['LAESTAB','School name', 'School postcode', '00_rank_band']]
ks4_hist_df.rename(columns={'00_rank_band':"2000"}, inplace=True)
ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)


In [ ]:
#year01 rank

ks4_2003 = ks4_2003.dropna(subset=["gcse_01"])
ks4_2003['01_rank'] = ks4_2003['gcse_01'].rank(ascending=False)
ks4_2003.sort_values('01_rank', ascending=True, inplace=True)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']
ks4_2003['01_rank_band'] = pd.cut(ks4_2003['01_rank'], bins=bins, labels=labels)

#building dataframe
ks4_2003.rename(columns={"institution name":"School name"},inplace=True)
ks4_2003.rename(columns={"institution postcode":"School postcode"},inplace=True)
ks4_2003_add = ks4_2003[['LAESTAB','School name', 'School postcode', '01_rank_band']]
ks4_2003_add.rename(columns={'01_rank_band':"2001"}, inplace=True)


# #left join to ks4_hist_df
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
#ks4_2003_add['LAESTAB'] = ks4_2003_add['LAESTAB'].astype(str)
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2003_add[['LAESTAB','2001']], how='left', on='LAESTAB')
#ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

#01 NAN: Bellerbys College (Closed in 2021, otherwise Indep)

,LAESTAB,School name,School postcode,2000,2001
0,8464605,Cardinal Newman Catholic School,BN3 6ND,Outstanding,Outstanding
1,8464012,Varndean School,BN1 6NP,Outstanding,Good
2,8464016,Dorothy Stringer High School,BN1 6PZ,Outstanding,Outstanding
3,8464018,Longhill High School,BN2 7FR,Good,Outstanding
4,8464067,Blatchington Mill School and Sixth Form College,BN3 7BW,Good,Good
5,8464068,Hove Park School and Sixth Form Centre,BN3 7BN,Good,Good
6,8464049,Portslade Community College,BN41 2WS,Poor,Poor
7,8464072,Patcham High School,BN1 8PB,Poor,Poor
8,8464022,Falmer High School,BN1 9PW,Poor,Poor


In [ ]:
## year02 rank

ks4_2003 = ks4_2003.dropna(subset=["gcse_02"])
ks4_2003['02_rank'] = ks4_2003['gcse_02'].rank(ascending=False)
ks4_2003.sort_values('02_rank', ascending=True, inplace=True)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']
ks4_2003['02_rank_band'] = pd.cut(ks4_2003['02_rank'], bins=bins, labels=labels)

#building dataframe
ks4_2003.rename(columns={"institution name":"School name"},inplace=True)
ks4_2003.rename(columns={"institution postcode":"School postcode"},inplace=True)
ks4_2003_add = ks4_2003[['LAESTAB','School name', 'School postcode', '02_rank_band']]
ks4_2003_add.rename(columns={'02_rank_band':"2002"}, inplace=True)


# #left join to ks4_hist_df
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
ks4_2003_add['LAESTAB'] = ks4_2003_add['LAESTAB'].astype(str)
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2003_add[['LAESTAB','2002']], how='left', on='LAESTAB')
#ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)
#02 NAN K-BIS Theatre School (Closed in 2015, otherwise Indep)

,LAESTAB,School name,School postcode,2000,2001,2002
0,8464605,Cardinal Newman Catholic School,BN3 6ND,Outstanding,Outstanding,Outstanding
1,8464012,Varndean School,BN1 6NP,Outstanding,Good,Outstanding
2,8464016,Dorothy Stringer High School,BN1 6PZ,Outstanding,Outstanding,Outstanding
3,8464018,Longhill High School,BN2 7FR,Good,Outstanding,Good
4,8464067,Blatchington Mill School and Sixth Form College,BN3 7BW,Good,Good,Good
5,8464068,Hove Park School and Sixth Form Centre,BN3 7BN,Good,Good,Good
6,8464049,Portslade Community College,BN41 2WS,Poor,Poor,Poor
7,8464072,Patcham High School,BN1 8PB,Poor,Poor,Poor
8,8464022,Falmer High School,BN1 9PW,Poor,Poor,Poor


In [13]:
## year02 rank

ks4_2003 = ks4_2003.dropna(subset=["gcse_03"])
ks4_2003['03_rank'] = ks4_2003['gcse_03'].rank(ascending=False)
ks4_2003.sort_values('03_rank', ascending=True, inplace=True)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']
ks4_2003['03_rank_band'] = pd.cut(ks4_2003['03_rank'], bins=bins, labels=labels)

#building dataframe
ks4_2003.rename(columns={"institution name":"School name"},inplace=True)
ks4_2003.rename(columns={"institution postcode":"School postcode"},inplace=True)
ks4_2003_add = ks4_2003[['LAESTAB','School name', 'School postcode', '03_rank_band']]
ks4_2003_add.rename(columns={'03_rank_band':"2003"}, inplace=True)


# #left join to ks4_hist_df
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
ks4_2003_add['LAESTAB'] = ks4_2003_add['LAESTAB'].astype(str)
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2003_add[['LAESTAB','2003']], how='left', on='LAESTAB')
#ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)
#03 NAN K-BIS and Bellerbys
## Varndean and Dorothy Stringer tied, manually changed both to Good rather than Outstanding. 
## Leaves us with 2 Outstanding 3 Good rather than 4 Outstanding 1 Good

### 2004

In [22]:
#### 2004
#read in xls
england_ks4_04 = pd.read_excel("schools/brighton/ks4/2003-2004_england_ks4final.xls", sheet_name=2, header=0)
#extract brighton schools
brighton_ks4_04 = england_ks4_04[england_ks4_04['LEA number']== 846]
brighton_ks4_04 = brighton_ks4_04[brighton_ks4_04['record type']== 1]
#extract key columns
ks4_2004 = brighton_ks4_04[['LEA number','ESTAB number','institution name','institution postcode',
                            '% of 15 year old pupils achieving 5 or more grades A*-C (GCSE or equivalent) - 2004 ']]
ks4_2004['LAESTAB'] = ks4_2004['LEA number'].astype(str) + ks4_2004['ESTAB number'].astype(str)
ks4_2004.rename(columns={"% of 15 year old pupils achieving 5 or more grades A*-C (GCSE or equivalent) - 2004 ":"gcse_04"},inplace=True)

ks4_2004['gcse_04'] = pd.to_numeric(ks4_2004['gcse_04'], errors='coerce')
# Bellerbys is 0. 
#read in ks4_hist_df, then left join 'gcse_04' to a new temp df derived from ks4_hist
# then use new temp df to rank and bin before adding to ks4_hist_df

ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
#year04 rank
ks4_2004 = ks4_2004.dropna(subset=["gcse_04"])
ks4_2004.rename(columns={"institution name":"School name"},inplace=True)
ks4_2004.rename(columns={"institution postcode":"School postcode"},inplace=True)
ks4_2004_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
#int/ str error: corrections
ks4_2004_add['LAESTAB'] = ks4_2004_add['LAESTAB'].astype(str)
ks4_2004_add = pd.merge(ks4_2004_add, ks4_2004[['LAESTAB','gcse_04']], how='left', on='LAESTAB')

ks4_2004_add['04_rank'] = ks4_2004_add['gcse_04'].rank(ascending=False)
ks4_2004_add.sort_values('04_rank', ascending=True, inplace=True)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']
ks4_2004_add['04_rank_band'] = pd.cut(ks4_2004_add['04_rank'], bins=bins, labels=labels)
ks4_2004_add.rename(columns={'04_rank_band':"2004"}, inplace=True)


# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2004_add[['LAESTAB','2004']], how='left', on='LAESTAB')
#ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)
# ADDED IN (manually to ks4_hist_df based on LAESTAB in ks4_2004)
#Drive Preparatory School (Other Indep School)
#Brighton Steiner School Limited finally added in after previous years being 0 

### 2005-2006

In [30]:
#### 2005
#read in xls
england_ks4_06 = pd.read_excel("schools/brighton/ks4/2005-2006_england_ks4final.xls", sheet_name=0, header=0)
#extract brighton schools
brighton_ks4_06 = england_ks4_06[england_ks4_06['LA number']== 846]
brighton_ks4_06 = brighton_ks4_06[brighton_ks4_06['record type']== 1]
#extract key columns
ks4_2006 = brighton_ks4_06[['LA number','ESTAB number','institution name','institution postcode',
                            '% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2005 ',
                            '% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2006']]
ks4_2006['LAESTAB'] = ks4_2006['LA number'].astype(str) + ks4_2006['ESTAB number'].astype(str)
ks4_2006.rename(columns={"% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2005 ":"gcse_05"},inplace=True)
ks4_2006.rename(columns={"% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2006":"gcse_06"},inplace=True)
ks4_2006['gcse_05'] = pd.to_numeric(ks4_2006['gcse_05'], errors='coerce')
ks4_2006['gcse_06'] = pd.to_numeric(ks4_2006['gcse_06'], errors='coerce')

#05 ranks
#read in ks4_hist_df, then left join 'gcse_05' to a new temp df derived from ks4_hist
# then use new temp df to rank and bin before adding to ks4_hist_df

ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
#year04 rank
ks4_2006 = ks4_2006.dropna(subset=["gcse_05"])
ks4_2006.rename(columns={"institution name":"School name"},inplace=True)
ks4_2006.rename(columns={"institution postcode":"School postcode"},inplace=True)
ks4_2006_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
#int/ str error: corrections
ks4_2006_add['LAESTAB'] = ks4_2006_add['LAESTAB'].astype(str)
ks4_2006_add = pd.merge(ks4_2006_add, ks4_2006[['LAESTAB','gcse_05']], how='left', on='LAESTAB')

ks4_2006_add['05_rank'] = ks4_2006_add['gcse_05'].rank(ascending=False)
ks4_2006_add.sort_values('05_rank', ascending=True, inplace=True)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']
ks4_2006_add['05_rank_band'] = pd.cut(ks4_2006_add['05_rank'], bins=bins, labels=labels)
ks4_2006_add.rename(columns={'05_rank_band':"2005"}, inplace=True)


# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2006_add[['LAESTAB','2005']], how='left', on='LAESTAB')
#ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)


# 2005 NAN
#East Brighton College of Media Arts --> no info

In [ ]:
#### 05 ranks
#read in ks4_hist_df, then left join 'gcse_05' to a new temp df derived from ks4_hist
# then use new temp df to rank and bin before adding to ks4_hist_df

ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
#year04 rank
ks4_2006 = ks4_2006.dropna(subset=["gcse_06"])
ks4_2006.rename(columns={"institution name":"School name"},inplace=True)
ks4_2006.rename(columns={"institution postcode":"School postcode"},inplace=True)
ks4_2006_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
#int/ str error: corrections
ks4_2006_add['LAESTAB'] = ks4_2006_add['LAESTAB'].astype(str)
ks4_2006_add = pd.merge(ks4_2006_add, ks4_2006[['LAESTAB','gcse_06']], how='left', on='LAESTAB')

ks4_2006_add['06_rank'] = ks4_2006_add['gcse_06'].rank(ascending=False)
ks4_2006_add.sort_values('06_rank', ascending=True, inplace=True)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']
ks4_2006_add['06_rank_band'] = pd.cut(ks4_2006_add['06_rank'], bins=bins, labels=labels)
ks4_2006_add.rename(columns={'06_rank_band':"2006"}, inplace=True)


# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2006_add[['LAESTAB','2006']], how='left', on='LAESTAB')
#ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

# 2006 NAN
#Bellerbys Nan
#East Brighton College of Media Arts still NaN

## 2007-2011 GCSEs and OFSR

In [2]:
#OFSTED Rankings
ofsr_long = pd.read_csv("schools/brighton/OFSR_hist_long.csv")

#year_list = ofsr_long['YEAR'].unique()
ofsr_wide = ofsr_long.pivot(index='SCHNAME', columns='YEAR', values='OFSR')
ofsr_wide.reset_index(inplace=True)

In [69]:
# #### 2007
# # #read in xls
# england_ks4_08 = pd.read_excel("schools/brighton/ks4/2007-2008_england_ks4final.xls", sheet_name=2, header=0)
# #extract brighton schools
# brighton_ks4_08 = england_ks4_08[england_ks4_08['LA number']== 846]
# brighton_ks4_08 = brighton_ks4_08[brighton_ks4_08['record type']== 1]
# #extract key columns
# ks4_2007 = brighton_ks4_08[['LA number','ESTAB number','institution name','institution postcode',
#                             '% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2007 ']]
# ks4_2007['LAESTAB'] = ks4_2007['LA number'].astype(str) + ks4_2007['ESTAB number'].astype(str)
# ks4_2007.rename(columns={"% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2007 ":"gcse_07"},inplace=True)
# ks4_2007['gcse_07'] = pd.to_numeric(ks4_2007['gcse_07'], errors='coerce')

# #'% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2008'
# #ks4_2008.rename(columns={"% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2008":"gcse_08"},inplace=True)
# #year07 acad rank
# ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# ks4_2007 = ks4_2007.dropna(subset=["gcse_07"])
# ks4_2007.rename(columns={"institution name":"School name"},inplace=True)
# ks4_2007.rename(columns={"institution postcode":"School postcode"},inplace=True)
# ks4_2007_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
# #int/ str error: corrections
# ks4_2007_add['LAESTAB'] = ks4_2007_add['LAESTAB'].astype(str)
# ks4_2007_add = pd.merge(ks4_2007_add, ks4_2007[['LAESTAB','gcse_07']], how='left', on='LAESTAB')

# ks4_2007_add['07_acad_rank'] = ks4_2007_add['gcse_07'].rank(ascending=False)
# ks4_2007_add.sort_values('07_acad_rank', ascending=True, inplace=True)

# # #merge 2007 ks4 acad results with 2007 ofsted rating
# ks4_2007_add = pd.merge(ks4_2007_add, ofsr_wide[['SCHNAME',2007]], how='left', left_on='School name', right_on='SCHNAME')
# ks4_2007_add.rename(columns={2007:"07_ofsr"}, inplace=True)

# ## export to missing csv to add in ofsr
# ks4_2007_add.to_csv('schools/brighton/2007-2011/2006-2007_brighton_ks4_missing.csv', index=False)

## Step 2 for 2007
ks4_2007_add = pd.read_csv('schools/brighton/2007-2011/2006-2007_brighton_ks4_completed.csv')
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2007_add['ofsr_values'] = ks4_2007_add['07_ofsr'].map(mapping)

#year07 complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks4_2007_add['07_rank'] = (ks4_2007_add['07_acad_rank'].rank(ascending=True) * 0.5 +
                    ks4_2007_add['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']

#building dataframe
ks4_2007_add['07_rank_band'] = pd.qcut(ks4_2007_add['07_rank'], q=3 , labels=labels)
ks4_2007_add.rename(columns={'07_rank_band':"2007"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2007_add['LAESTAB'] = ks4_2007_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2007_add[['LAESTAB','2007']], how='left', on='LAESTAB')
#ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)



###Notes previously
# 2007 missing schools: 
# Brighton and Hove High School --> Brighton Girls GDST (Indep) --> no ofsted --> drop
# Blatchington Mill School (and Sixth Form) --> has ofsted
# St Mary's Hall merged with Roedean in 2009 --> no oftsed --> drop
# Roedean independent too --> no ofsted --> drop
# Portslade --> PACA --> has ofsted
# Falmer --> BACA --> has ofsted
# Brighton Steiner --> Waldorf --> has ofsted surprisngly --> drop?
# Bellerbys College --> independent --> no ofsted --> drop? 


In [5]:
# 2008
# # #read in xls
# england_ks4_08 = pd.read_excel("schools/brighton/ks4/2007-2008_england_ks4final.xls", sheet_name=2, header=0)
# #extract brighton schools
# brighton_ks4_08 = england_ks4_08[england_ks4_08['LA number']== 846]
# brighton_ks4_08 = brighton_ks4_08[brighton_ks4_08['record type']== 1]
# #extract key columns
# ks4_2008 = brighton_ks4_08[['LA number','ESTAB number','institution name','institution postcode',
#                             '% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2008']]
# ks4_2008['LAESTAB'] = ks4_2008['LA number'].astype(str) + ks4_2008['ESTAB number'].astype(str)
# ks4_2008.rename(columns={"% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2008":"gcse_08"},inplace=True)
# ks4_2008['gcse_08'] = pd.to_numeric(ks4_2008['gcse_08'], errors='coerce')

# #'% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2008'
# #ks4_2008.rename(columns={"% of 15 year old pupils achieving Level 2 threshold (GCSE and Equivalent) - 2008":"gcse_08"},inplace=True)
# #year08 acad rank
# ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# ks4_2008 = ks4_2008.dropna(subset=["gcse_08"])
# ks4_2008.rename(columns={"institution name":"School name"},inplace=True)
# ks4_2008.rename(columns={"institution postcode":"School postcode"},inplace=True)
# ks4_2008_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
# #int/ str error: corrections
# ks4_2008_add['LAESTAB'] = ks4_2008_add['LAESTAB'].astype(str)
# ks4_2008_add = pd.merge(ks4_2008_add, ks4_2008[['LAESTAB','gcse_08']], how='left', on='LAESTAB')

# ks4_2008_add['08_acad_rank'] = ks4_2008_add['gcse_08'].rank(ascending=False)
# ks4_2008_add.sort_values('08_acad_rank', ascending=True, inplace=True)

# # #merge 2008 ks4 acad results with 2008 ofsted rating
# ks4_2008_add = pd.merge(ks4_2008_add, ofsr_wide[['SCHNAME',2008]], how='left', left_on='School name', right_on='SCHNAME')
# ks4_2008_add.rename(columns={2008:"08_ofsr"}, inplace=True)

# ## export to missing csv to add in ofsr
# ks4_2008_add.to_csv('schools/brighton/2007-2011/2007-2008_brighton_ks4_missing.csv', index=False)

## Step 2 for 2008
ks4_2008_add = pd.read_csv('schools/brighton/2007-2011/2007-2008_brighton_ks4_completed.csv')
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2008_add['ofsr_values'] = ks4_2008_add['08_ofsr'].map(mapping)

#year07 complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks4_2008_add['08_rank'] = (ks4_2008_add['08_acad_rank'].rank(ascending=True) * 0.5 +
                    ks4_2008_add['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']

#building dataframe
ks4_2008_add['08_rank_band'] = pd.qcut(ks4_2008_add['08_rank'], q=3 , labels=labels)
ks4_2008_add.rename(columns={'08_rank_band':"2008"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2008_add['LAESTAB'] = ks4_2008_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2008_add[['LAESTAB','2008']], how='left', on='LAESTAB')
#ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

In [9]:
# ###2009
# # #read in xls
# england_ks4_09 = pd.read_excel("schools/brighton/ks4/2008-2009_england_ks4final.xls", sheet_name=2, header=0)
# #extract brighton schools
# brighton_ks4_09 = england_ks4_09[england_ks4_09['LA number']== 846]
# brighton_ks4_09 = brighton_ks4_09[brighton_ks4_09['record type']== 1]
# #extract key columns
# ks4_2009 = brighton_ks4_09[['LA number','ESTAB number','institution name','institution postcode',
#                             '% of pupils achieving Level 2 threshold']]
# ks4_2009['LAESTAB'] = ks4_2009['LA number'].astype(str) + ks4_2009['ESTAB number'].astype(str)
# ks4_2009.rename(columns={"% of pupils achieving Level 2 threshold":"gcse_09"},inplace=True)
# ks4_2009['gcse_09'] = pd.to_numeric(ks4_2009['gcse_09'], errors='coerce')


# #year09 acad rank
# ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# ks4_2009 = ks4_2009.dropna(subset=["gcse_09"])
# ks4_2009.rename(columns={"institution name":"School name"},inplace=True)
# ks4_2009.rename(columns={"institution postcode":"School postcode"},inplace=True)
# ks4_2009_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
# #int/ str error: corrections
# ks4_2009_add['LAESTAB'] = ks4_2009_add['LAESTAB'].astype(str)
# ks4_2009_add = pd.merge(ks4_2009_add, ks4_2009[['LAESTAB','gcse_09']], how='left', on='LAESTAB')

# ks4_2009_add['09_acad_rank'] = ks4_2009_add['gcse_09'].rank(ascending=False)
# ks4_2009_add.sort_values('09_acad_rank', ascending=True, inplace=True)

# # #merge ks4 acad results with  ofsted rating
# ks4_2009_add = pd.merge(ks4_2009_add, ofsr_wide[['SCHNAME',2009]], how='left', left_on='School name', right_on='SCHNAME')
# ks4_2009_add.rename(columns={2009:"09_ofsr"}, inplace=True)

# ## export to missing csv to add in ofsr
# ks4_2009_add.to_csv('schools/brighton/2007-2011/2008-2009_brighton_ks4_missing.csv', index=False)

## Step 2 
ks4_2009_add = pd.read_csv('schools/brighton/2007-2011/2008-2009_brighton_ks4_completed.csv')
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2009_add['ofsr_values'] = ks4_2009_add['09_ofsr'].map(mapping)

#year07 complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks4_2009_add['09_rank'] = (ks4_2009_add['09_acad_rank'].rank(ascending=True) * 0.5 +
                    ks4_2009_add['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']

#building dataframe
ks4_2009_add['09_rank_band'] = pd.qcut(ks4_2009_add['09_rank'], q=3 , labels=labels)
ks4_2009_add.rename(columns={'09_rank_band':"2009"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2009_add['LAESTAB'] = ks4_2009_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2009_add[['LAESTAB','2009']], how='left', on='LAESTAB')
# #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

In [10]:
###2010
# # #read in xls
# england_ks4_10 = pd.read_excel("schools/brighton/ks4/2009-2010_england_ks4final.xls", sheet_name=0, header=0)
# #extract brighton schools
# brighton_ks4_10 = england_ks4_10[england_ks4_10['LEA']== 846]
# brighton_ks4_10 = brighton_ks4_10[brighton_ks4_10['RECTYPE']== 1]
# #extract key columns
# ks4_2010 = brighton_ks4_10[['LEA','ESTAB','SCHNAME','PCODE',
#                             'PTAC5EM']]
# ks4_2010['LEA'] = ks4_2010['LEA'].astype(int) 
# ks4_2010['ESTAB'] = ks4_2010['ESTAB'].astype(int)
# ks4_2010['LAESTAB'] = ks4_2010['LEA'].astype(str) + ks4_2010['ESTAB'].astype(str)
# ks4_2010['PTAC5EM'] = pd.to_numeric(ks4_2010['PTAC5EM'], errors='coerce')
# ks4_2010 = ks4_2010.dropna(subset=["PTAC5EM"])
# ks4_2010.rename(columns={"PTAC5EM":"gcse_10"},inplace=True)
# ks4_2010['gcse_10'] = pd.to_numeric(ks4_2010['gcse_10'], errors='coerce')

# #year10 acad rank
# ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# ks4_2010.rename(columns={"institution name":"School name"},inplace=True)
# ks4_2010.rename(columns={"institution postcode":"School postcode"},inplace=True)
# ks4_2010_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
# #int/ str error: corrections
# ks4_2010_add['LAESTAB'] = ks4_2010_add['LAESTAB'].astype(str)
# ks4_2010_add = pd.merge(ks4_2010_add, ks4_2010[['LAESTAB','gcse_10']], how='left', on='LAESTAB')

# ks4_2010_add['10_acad_rank'] = ks4_2010_add['gcse_10'].rank(ascending=False)
# ks4_2010_add.sort_values('10_acad_rank', ascending=True, inplace=True)

# # #merge ks4 acad results with  ofsted rating
# ks4_2010_add = pd.merge(ks4_2010_add, ofsr_wide[['SCHNAME',2010]], how='left', left_on='School name', right_on='SCHNAME')
# ks4_2010_add.rename(columns={2010:"10_ofsr"}, inplace=True)

# ## export to missing csv to add in ofsr
# ks4_2010_add.to_csv('schools/brighton/2007-2011/2009-2010_brighton_ks4_missing.csv', index=False)

# ## Step 2 
ks4_2010_add = pd.read_csv('schools/brighton/2007-2011/2009-2010_brighton_ks4_completed.csv')
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2010_add['ofsr_values'] = ks4_2010_add['10_ofsr'].map(mapping)

#complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks4_2010_add['10_rank'] = (ks4_2010_add['10_acad_rank'].rank(ascending=True) * 0.5 +
                    ks4_2010_add['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']

#building dataframe
ks4_2010_add['10_rank_band'] = pd.qcut(ks4_2010_add['10_rank'], q=3 , labels=labels)
ks4_2010_add.rename(columns={'10_rank_band':"2010"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2010_add['LAESTAB'] = ks4_2010_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2010_add[['LAESTAB','2010']], how='left', on='LAESTAB')
#ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

In [40]:
### 2011
# ## 2011 data: BACA replaces Falmer, manually changed Falmer in ks4 hist df to BACA for 2011 merging
# #read in brighton schools
# brighton_ks4_11 = pd.read_csv("schools/brighton/2007-2011/2010-2011_846_ks4final.csv")
# brighton_ks4_11 = brighton_ks4_11[brighton_ks4_11['RECTYPE']== 1]

# #extract key columns
# ks4_2011 = brighton_ks4_11[['LEA','ESTAB','SCHNAME', 'PTAC5EM']]
# ks4_2011['LEA'] = ks4_2011['LEA'].astype(int) 
# ks4_2011['ESTAB'] = ks4_2011['ESTAB'].astype(int)
# ks4_2011['LAESTAB'] = ks4_2011['LEA'].astype(str) + ks4_2011['ESTAB'].astype(str)
# #ks4_2011['PTAC5EM'] = pd.to_numeric(ks4_2011['PTAC5EM'], errors='coerce')
# #ks4_2011 = ks4_2011.dropna(subset=["PTAC5EM"])
# ks4_2011['PTAC5EM'] = ks4_2011['PTAC5EM'].str.rstrip("%").dropna().astype(float)
# ks4_2011.rename(columns={"PTAC5EM":"gcse_11"},inplace=True)
# #ks4_2011['gcse_11'] = pd.to_numeric(ks4_2011['gcse_11'], errors='coerce')

# ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# ks4_2011.rename(columns={"SCHNAME":"School name"},inplace=True)
# ks4_2011_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
# #int/ str error: corrections
# ks4_2011_add['LAESTAB'] = ks4_2011_add['LAESTAB'].astype(str)
# ks4_2011_add = pd.merge(ks4_2011_add, ks4_2011[['LAESTAB','gcse_11']], how='left', on='LAESTAB')
# ks4_2011_add['11_acad_rank'] = ks4_2011_add['gcse_11'].rank(ascending=False)
# ks4_2011_add.sort_values('11_acad_rank', ascending=True, inplace=True)

# # #merge ks4 acad results with  ofsted rating
# ks4_2011_add = pd.merge(ks4_2011_add, ofsr_wide[['SCHNAME',2011]], how='left', left_on='School name', right_on='SCHNAME')
# ks4_2011_add.rename(columns={2011:"11_ofsr"}, inplace=True)

# ## export to missing csv to add in ofsr
# ks4_2011_add.to_csv('schools/brighton/2007-2011/2010-2011_brighton_ks4_missing.csv', index=False)

## Step 2 
ks4_2011_add = pd.read_csv('schools/brighton/2007-2011/2010-2011_brighton_ks4_completed.csv')
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2011_add['ofsr_values'] = ks4_2011_add['11_ofsr'].map(mapping)

#complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
ks4_2011_add['11_rank'] = (ks4_2011_add['11_acad_rank'].rank(ascending=True) * 0.5 +
                    ks4_2011_add['ofsr_values'].rank(ascending=True, method='dense') * 0.5)
bins = 3
labels = ['Outstanding', 'Good', 'Poor']

#building dataframe
ks4_2011_add['11_rank_band'] = pd.qcut(ks4_2011_add['11_rank'], q=3 , labels=labels)
ks4_2011_add.rename(columns={'11_rank_band':"2011"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2011_add['LAESTAB'] = ks4_2011_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2011_add[['LAESTAB','2011']], how='left', on='LAESTAB')
# #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

### Note 2011 4 schools outstanding (DS, CN, BM, Varndean) --> BM and Vardndean tie over gcse and ofsted 

## 2012 onwards: GCSE, OFSR, FSM and pupdest

In [66]:
# #read in csv

# brighton_ks4_12 = pd.read_csv("schools/brighton/2012-2019/2011-2012_846_ks4final.csv")
# brighton_ks4_12 = brighton_ks4_12[brighton_ks4_12['RECTYPE']== 1]

# #extract key columns
# ks4_2012 = brighton_ks4_12[['LEA','ESTAB', 'URN', 'SCHNAME', 'PTAC5EM']]

# ks4_2012['LEA'] = ks4_2012['LEA'].astype(int) 
# ks4_2012['ESTAB'] = ks4_2012['ESTAB'].astype(int)
# ks4_2012['LAESTAB'] = ks4_2012['LEA'].astype(str) + ks4_2012['ESTAB'].astype(str)


# ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# ks4_2012_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
# #int/ str error: corrections
# ks4_2012_add['LAESTAB'] = ks4_2012_add['LAESTAB'].astype(str)
# ks4_2012_add = pd.merge(ks4_2012_add, ks4_2012[['LAESTAB', 'URN','PTAC5EM']], how='left', on='LAESTAB')
# ks4_2012_add['PTAC5EM'] = ks4_2012_add['PTAC5EM'].str.rstrip("%").dropna().astype(float)
# ks4_2012_add.rename(columns={"PTAC5EM":"gcse_12"},inplace=True)
# ks4_2012_add.rename(columns={"SCHNAME":"School name"},inplace=True)
# ks4_2012_add['12_acad_rank'] = ks4_2012_add['gcse_12'].rank(ascending=False)
# ks4_2012_add.sort_values('12_acad_rank', ascending=True, inplace=True)

# # #merge ks4 acad results with  ofsted rating
# ks4_2012_add = pd.merge(ks4_2012_add, ofsr_wide[['SCHNAME',2012]], how='left', left_on='School name', right_on='SCHNAME')
# ks4_2012_add.rename(columns={2012:"12_ofsr"}, inplace=True)

# ## export to missing csv to add in ofsr
# ks4_2012_add.to_csv('schools/brighton/2012-2019/2011-2012_brighton_ks4_missing.csv', index=False)

## Step 2 add  FSM and pupdest
ks4_2012_add = pd.read_csv('schools/brighton/2012-2019/2011-2012_brighton_ks4_completed.csv')


#fsm 
fsm_2012 = pd.read_csv('schools/brighton/fsm/2011-2012_846_cfr.csv')
fsm_2012['URN'] = pd.to_numeric(fsm_2012['URN'], errors='coerce').dropna().astype(int)
ks4_2012_add = pd.merge(ks4_2012_add, fsm_2012[['URN','FSM']], how='left', on='URN')

#pupdest
pupd_2012 = pd.read_csv('schools/brighton/pupdest/2011-2012_846_pupdest.csv')
pupd_2012['URN'] = pd.to_numeric(pupd_2012['URN'], errors='coerce').dropna().astype(int)
pupd_2012['SSFPER'] = pupd_2012['SSFPER'].str.rstrip("%").dropna().astype(float)
pupd_2012['SFCPER'] = pupd_2012['SFCPER'].str.rstrip("%").dropna().astype(float)
pupd_2012['SF_DEST'] = pupd_2012['SSFPER'] + pupd_2012['SFCPER']
ks4_2012_add = pd.merge(ks4_2012_add, pupd_2012[['URN','SF_DEST']], how='left', on='URN')

#get ready to merge
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2012_add['ofsr_values'] = ks4_2012_add['12_ofsr'].map(mapping)

default_rank = 9  # Choose an appropriate default rank value
ks4_2012_add['12_rank'] = default_rank

#complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
# ks4_2012_add['12_rank'] = (ks4_2012_add['12_acad_rank'].rank(ascending=True) * 0.3 + 
#                            ks4_2012_add['ofsr_values'].rank(ascending=True, method='dense') * 0.3 + 
#                            ks4_2012_add['SF_DEST'].rank(ascending=False) * 0.3 + 
#                            ks4_2012_add['FSM'].rank(ascending=True) * 0.1
#                           )

#test code
has_data = ks4_2012_add['FSM'].notnull() & ks4_2012_add['SF_DEST'].notnull()
ks4_2012_add.loc[has_data, '12_rank'] = (
    ks4_2012_add.loc[has_data, '12_acad_rank'].rank(ascending=True) * 0.3 +
    ks4_2012_add.loc[has_data, 'ofsr_values'].rank(ascending=True, method='dense') * 0.3 +
    ks4_2012_add.loc[has_data, 'SF_DEST'].rank(ascending=False) * 0.3 +
    ks4_2012_add.loc[has_data, 'FSM'].rank(ascending=True) * 0.1
)


bins = 3
labels = ['Outstanding', 'Good', 'Poor']

#building dataframe
ks4_2012_add['12_rank_band'] = pd.qcut(ks4_2012_add['12_rank'], q=3 , labels=labels)
ks4_2012_add.rename(columns={'12_rank_band':"2012"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2012_add['LAESTAB'] = ks4_2012_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2012_add[['LAESTAB','2012']], how='left', on='LAESTAB')


# #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)
#### 2012: Portslade no GCSE data
#### PACA and BACA no pupdest nor FSM data 

In [17]:
# 2013
# read in csv

brighton_ks4_13 = pd.read_csv("schools/brighton/2012-2019/2012-2013_846_ks4final.csv")
brighton_ks4_13 = brighton_ks4_13[brighton_ks4_13['RECTYPE']== 1]

#extract key columns
ks4_2013 = brighton_ks4_13[['LEA','ESTAB', 'URN', 'SCHNAME', 'PTAC5EM']]

ks4_2013['LEA'] = ks4_2013['LEA'].astype(int) 
ks4_2013['ESTAB'] = ks4_2013['ESTAB'].astype(int)
ks4_2013['LAESTAB'] = ks4_2013['LEA'].astype(str) + ks4_2013['ESTAB'].astype(str)


ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
ks4_2013_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
#int/ str error: corrections
ks4_2013_add['LAESTAB'] = ks4_2013_add['LAESTAB'].astype(str)
ks4_2013_add = pd.merge(ks4_2013_add, ks4_2013[['LAESTAB', 'URN','PTAC5EM']], how='left', on='LAESTAB')
ks4_2013_add['PTAC5EM'] = ks4_2013_add['PTAC5EM'].str.rstrip("%").dropna().astype(float)
ks4_2013_add.rename(columns={"PTAC5EM":"gcse_13"},inplace=True)
ks4_2013_add.rename(columns={"SCHNAME":"School name"},inplace=True)
ks4_2013_add['13_acad_rank'] = ks4_2013_add['gcse_13'].rank(ascending=False)
ks4_2013_add.sort_values('13_acad_rank', ascending=True, inplace=True)

# #merge ks4 acad results with  ofsted rating
ks4_2013_add = pd.merge(ks4_2013_add, ofsr_wide[['SCHNAME',2013]], how='left', left_on='School name', right_on='SCHNAME')
ks4_2013_add.rename(columns={2013:"13_ofsr"}, inplace=True)

## export to missing csv to add in ofsr
ks4_2013_add.to_csv('schools/brighton/2012-2019/2012-2013_brighton_ks4_missing.csv', index=False)

# ## Step 2 add  FSM and pupdest
# ks4_2013_add = pd.read_csv('schools/brighton/2012-2019/2012-2013_brighton_ks4_completed.csv')
# ks4_2013_add['URN'] = ks4_2013_add['URN'].astype(int)

# #fsm 
# fsm_2013 = pd.read_csv('schools/brighton/fsm/2012-2013_846_cfr.csv')
# fsm_2013['URN'] = pd.to_numeric(fsm_2013['URN'], errors='coerce').dropna().astype(int)
# ks4_2013_add = pd.merge(ks4_2013_add, fsm_2013[['URN','FSM']], how='left', on='URN')
# ks4_2013_add
# #pupdest
# pupd_2013 = pd.read_csv('schools/brighton/pupdest/2012-2013_846_pupdest.csv')
# pupd_2013['URN'] = pd.to_numeric(pupd_2013['URN'], errors='coerce').dropna()
# pupd_2013['SSFPER'] = pupd_2013['SSFPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
# pupd_2013['SFCPER'] = pupd_2013['SFCPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
# pupd_2013['SF_DEST'] = pupd_2013['SSFPER'] + pupd_2013['SFCPER']

# ks4_2013_add = pd.merge(ks4_2013_add, pupd_2013[['URN','SF_DEST']], how='left', on='URN')


# #get ready to merge
# ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# # Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# # due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
# mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
# ks4_2013_add['ofsr_values'] = ks4_2013_add['13_ofsr'].map(mapping)


# default_rank = 9  # Choose an appropriate default rank value
# ks4_2013_add['13_rank'] = default_rank

# #complete rank
# #using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
# has_data = ks4_2013_add['FSM'].notnull() & ks4_2013_add['SF_DEST'].notnull()
# ks4_2013_add.loc[has_data, '13_rank'] = (
#     ks4_2013_add.loc[has_data, '13_acad_rank'].rank(ascending=True) * 0.3 +
#     ks4_2013_add.loc[has_data, 'ofsr_values'].rank(ascending=True, method='dense') * 0.3 +
#     ks4_2013_add.loc[has_data, 'SF_DEST'].rank(ascending=False) * 0.3 +
#     ks4_2013_add.loc[has_data, 'FSM'].rank(ascending=True) * 0.1
# )


# bins = 3
# labels = ['Outstanding', 'Good', 'Poor']


# #building dataframe
# ks4_2013_add['13_rank_band'] = pd.qcut(ks4_2013_add['13_rank'], q=3 , labels=labels)
# ks4_2013_add.rename(columns={'13_rank_band':"2013"}, inplace=True)

# # #left join to ks2_hist_df
# ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
# ks4_2013_add['LAESTAB'] = ks4_2013_add['LAESTAB'].astype(str)
# ks4_hist_df = pd.merge(ks4_hist_df, ks4_2013_add[['LAESTAB','2013']], how='left', on='LAESTAB')


# # #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)


#### Replace supp data with 0 
### BACA no fsm
### PACA no gcse, fsm, pupdest

In [9]:
## 2014
#read in csv
brighton_ks4_14 = pd.read_csv("schools/brighton/2012-2019/2013-2014_846_ks4final.csv")
brighton_ks4_14 = brighton_ks4_14[brighton_ks4_14['RECTYPE']== 1]

#extract key columns
### PTAC5EM_PTQ for 2014, PTAC5EM_PTQ_EE for 2015,2016
### but essentially Percentage of pupils achieving 5+ A*-C or equivalents including A*-C in both English and mathematics GCSEs
ks4_2014 = brighton_ks4_14[['LEA','ESTAB', 'URN', 'SCHNAME', 'PTAC5EM_PTQ']]

ks4_2014['LEA'] = ks4_2014['LEA'].astype(int) 
ks4_2014['ESTAB'] = ks4_2014['ESTAB'].astype(int)
ks4_2014['LAESTAB'] = ks4_2014['LEA'].astype(str) + ks4_2014['ESTAB'].astype(str)


ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
ks4_2014_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
#int/ str error: corrections
ks4_2014_add['LAESTAB'] = ks4_2014_add['LAESTAB'].astype(str)
ks4_2014_add = pd.merge(ks4_2014_add, ks4_2014[['LAESTAB', 'URN','PTAC5EM_PTQ']], how='left', on='LAESTAB')
ks4_2014_add['PTAC5EM_PTQ'] = ks4_2014_add['PTAC5EM_PTQ'].str.rstrip("%").dropna().astype(float)
ks4_2014_add.rename(columns={"PTAC5EM_PTQ":"gcse_14"},inplace=True)
ks4_2014_add.rename(columns={"SCHNAME":"School name"},inplace=True)

# #merge ks4 acad results with  ofsted rating
ks4_2014_add = pd.merge(ks4_2014_add, ofsr_wide[['SCHNAME',2014]], how='left', left_on='School name', right_on='SCHNAME')
ks4_2014_add.rename(columns={2014:"14_ofsr"}, inplace=True)

## export to missing csv to add in ofsr
ks4_2014_add.to_csv('schools/brighton/2012-2019/2013-2014_brighton_ks4_missing.csv', index=False)

# ## Step 2 add acad ranking (manually input PACA data found from 2015), FSM and pupdest
# ks4_2014_add = pd.read_csv('schools/brighton/2012-2019/2013-2014_brighton_ks4_completed.csv')
# ks4_2014_add['URN'] = ks4_2014_add['URN'].astype(int)

## acad ranking
# ks4_2014_add['14_acad_rank'] = ks4_2014_add['gcse_14'].rank(ascending=False)
# ks4_2014_add.sort_values('14_acad_rank', ascending=True, inplace=True)

# #fsm 
# fsm_2014 = pd.read_csv('schools/brighton/fsm/2013-2014_846_cfr.csv')
# fsm_2014['URN'] = pd.to_numeric(fsm_2014['URN'], errors='coerce').dropna().astype(int)
# ks4_2014_add = pd.merge(ks4_2014_add, fsm_2014[['URN','FSM']], how='left', on='URN')
# ks4_2014_add
# #pupdest
# pupd_2014 = pd.read_csv('schools/brighton/pupdest/2013-2014_846_pupdest.csv')
# pupd_2014['URN'] = pd.to_numeric(pupd_2014['URN'], errors='coerce').dropna()
# pupd_2014['SSFPER'] = pupd_2014['SSFPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
# pupd_2014['SFCPER'] = pupd_2014['SFCPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
# pupd_2014['SF_DEST'] = pupd_2014['SSFPER'] + pupd_2014['SFCPER']

# ks4_2014_add = pd.merge(ks4_2014_add, pupd_2014[['URN','SF_DEST']], how='left', on='URN')


# #get ready to merge
# ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# # Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# # due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
# mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
# ks4_2014_add['ofsr_values'] = ks4_2014_add['14_ofsr'].map(mapping)


# default_rank = 9  # Choose an appropriate default rank value
# ks4_2014_add['14_rank'] = default_rank

# #complete rank
# #using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)
# has_data = ks4_2014_add['FSM'].notnull() & ks4_2014_add['SF_DEST'].notnull()
# ks4_2014_add.loc[has_data, '14_rank'] = (
#     ks4_2014_add.loc[has_data, '14_acad_rank'].rank(ascending=True) * 0.3 +
#     ks4_2014_add.loc[has_data, 'ofsr_values'].rank(ascending=True, method='dense') * 0.3 +
#     ks4_2014_add.loc[has_data, 'SF_DEST'].rank(ascending=False) * 0.3 +
#     ks4_2014_add.loc[has_data, 'FSM'].rank(ascending=True) * 0.1
# )


# bins = 3
# labels = ['Outstanding', 'Good', 'Poor']


# #building dataframe
# ks4_2014_add['14_rank_band'] = pd.qcut(ks4_2014_add['14_rank'], q=3 , labels=labels)
# ks4_2014_add.rename(columns={'14_rank_band':"2014"}, inplace=True)

# # #left join to ks2_hist_df
# ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
# ks4_2014_add['LAESTAB'] = ks4_2014_add['LAESTAB'].astype(str)
# ks4_hist_df = pd.merge(ks4_hist_df, ks4_2014_add[['LAESTAB','2014']], how='left', on='LAESTAB')


# #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

In [26]:
### 2015
### manually changed Dorothy, PACA entry in ks4_hist_df to match new names (Blatchington M not yet)
### 2015 onwards full proper ranking, BACA and PACA fsm data assume 2023 
# #read in csv
# brighton_ks4_15 = pd.read_csv("schools/brighton/2012-2019/2014-2015_846_ks4final.csv")
# brighton_ks4_15 = brighton_ks4_15[brighton_ks4_15['RECTYPE']== 1]

# #extract key columns
# ### PTAC5EM_PTQ for 2014, PTAC5EM_PTQ_EE for 2015,2016
# ### but essentially Percentage of pupils achieving 5+ A*-C or equivalents including A*-C in both English and mathematics GCSEs
# ks4_2015 = brighton_ks4_15[['LEA','ESTAB', 'URN', 'SCHNAME', 'PTAC5EM_PTQ_EE']]
# ks4_2015['LEA'] = ks4_2015['LEA'].astype(int) 
# ks4_2015['ESTAB'] = ks4_2015['ESTAB'].astype(int)
# ks4_2015['LAESTAB'] = ks4_2015['LEA'].astype(str) + ks4_2015['ESTAB'].astype(str)

# ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# ks4_2015_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
# #int/ str error: corrections
# ks4_2015_add['LAESTAB'] = ks4_2015_add['LAESTAB'].astype(str)
# ks4_2015_add = pd.merge(ks4_2015_add, ks4_2015[['LAESTAB', 'URN','PTAC5EM_PTQ_EE']], how='left', on='LAESTAB')
# #int/ str error: corrections
# ks4_2015_add['PTAC5EM_PTQ_EE'] = ks4_2015_add['PTAC5EM_PTQ_EE'].str.rstrip("%").dropna().astype(float)
# ks4_2015_add.rename(columns={"PTAC5EM_PTQ_EE":"gcse_15"},inplace=True)
# ks4_2015_add.rename(columns={"SCHNAME":"School name"},inplace=True)

# # #merge ks4 acad results with  ofsted rating
# ks4_2015_add = pd.merge(ks4_2015_add, ofsr_wide[['SCHNAME',2015]], how='left', left_on='School name', right_on='SCHNAME')
# ks4_2015_add.rename(columns={2015:"15_ofsr"}, inplace=True)

# ## export to missing csv to add in ofsr
# ks4_2015_add.to_csv('schools/brighton/2012-2019/2014-2015_brighton_ks4_missing.csv', index=False)

## Step 2 add acad ranking (manually input PACA data found from 2015), FSM and pupdest
ks4_2015_add = pd.read_csv('schools/brighton/2012-2019/2014-2015_brighton_ks4_completed.csv')
ks4_2015_add['URN'] = ks4_2015_add['URN'].astype(int)

# acad ranking
ks4_2015_add['15_acad_rank'] = ks4_2015_add['gcse_15'].rank(ascending=False)
ks4_2015_add.sort_values('15_acad_rank', ascending=True, inplace=True)

#fsm 
fsm_2015 = pd.read_csv('schools/brighton/fsm/2014-2015_846_cfr.csv')
fsm_2015['URN'] = pd.to_numeric(fsm_2015['URN'], errors='coerce').dropna().astype(int)
ks4_2015_add = pd.merge(ks4_2015_add, fsm_2015[['URN','FSM']], how='left', on='URN')
#replace BACA with 44.7 (2023 value)
ks4_2015_add.loc[ks4_2015_add['URN'] == 136164, 'FSM'] = 44.7
#replace PACA with 25.6 (2023 value)
ks4_2015_add.loc[ks4_2015_add['URN'] == 137063, 'FSM'] = 25.6

#pupdest
pupd_2015 = pd.read_csv('schools/brighton/pupdest/2014-2015_846_pupdest.csv')
pupd_2015['URN'] = pd.to_numeric(pupd_2015['URN'], errors='coerce').dropna()
pupd_2015['SSFPER'] = pupd_2015['SSFPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2015['SFCPER'] = pupd_2015['SFCPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2015['SF_DEST'] = pupd_2015['SSFPER'] + pupd_2015['SFCPER']

ks4_2015_add = pd.merge(ks4_2015_add, pupd_2015[['URN','SF_DEST']], how='left', on='URN')


#get ready to merge
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2015_add['ofsr_values'] = ks4_2015_add['15_ofsr'].map(mapping)

#complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)

ks4_2015_add['15_rank'] = (ks4_2015_add['15_acad_rank'].rank(ascending=True) * 0.3 + 
                           ks4_2015_add['ofsr_values'].rank(ascending=True, method='dense') * 0.3 + 
                           ks4_2015_add['SF_DEST'].rank(ascending=False) * 0.3 + 
                           ks4_2015_add['FSM'].rank(ascending=True) * 0.1
                          )


bins = 3
labels = ['Outstanding', 'Good', 'Poor']


#building dataframe
ks4_2015_add['15_rank_band'] = pd.qcut(ks4_2015_add['15_rank'], q=3 , labels=labels)
ks4_2015_add.rename(columns={'15_rank_band':"2015"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2015_add['LAESTAB'] = ks4_2015_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2015_add[['LAESTAB','2015']], how='left', on='LAESTAB')

# #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

In [36]:
### 2016
### Blatchington M only changes name for 2019 data
### 2015 onwards full proper ranking, BACA and PACA fsm data assume 2023 
#read in csv
brighton_ks4_16 = pd.read_csv("schools/brighton/2012-2019/2015-2016_846_ks4final.csv")
brighton_ks4_16 = brighton_ks4_16[brighton_ks4_16['RECTYPE']== 1]

#extract key columns
### PTAC5EM_PTQ for 2014, PTAC5EM_PTQ_EE for 2015,2016
### but essentially Percentage of pupils achieving 5+ A*-C or equivalents including A*-C in both English and mathematics GCSEs
ks4_2016 = brighton_ks4_16[['LEA','ESTAB', 'URN', 'SCHNAME', 'PTAC5EM_PTQ_EE']]
ks4_2016['LEA'] = ks4_2016['LEA'].astype(int) 
ks4_2016['ESTAB'] = ks4_2016['ESTAB'].astype(int)
ks4_2016['LAESTAB'] = ks4_2016['LEA'].astype(str) + ks4_2016['ESTAB'].astype(str)

ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
ks4_2016_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
#int/ str error: corrections
ks4_2016_add['LAESTAB'] = ks4_2016_add['LAESTAB'].astype(str)
ks4_2016_add = pd.merge(ks4_2016_add, ks4_2016[['LAESTAB', 'URN','PTAC5EM_PTQ_EE']], how='left', on='LAESTAB')
#int/ str error: corrections
ks4_2016_add['PTAC5EM_PTQ_EE'] = ks4_2016_add['PTAC5EM_PTQ_EE'].str.rstrip("%").dropna().astype(float)
ks4_2016_add.rename(columns={"PTAC5EM_PTQ_EE":"gcse_16"},inplace=True)
ks4_2016_add.rename(columns={"SCHNAME":"School name"},inplace=True)

# #merge ks4 acad results with  ofsted rating
ks4_2016_add = pd.merge(ks4_2016_add, ofsr_wide[['SCHNAME',2016]], how='left', left_on='School name', right_on='SCHNAME')
ks4_2016_add.rename(columns={2016:"16_ofsr"}, inplace=True)
#replace Blatchington M with Good (from 2013-2022)
ks4_2016_add.loc[ks4_2016_add['URN'] == '114606', '16_ofsr'] = 'Good'
#### export to missing csv to add in ofsr ( no longer needed now)
##ks4_2016_add.to_csv('schools/brighton/2012-2019/2015-2016_brighton_ks4_completed.csv', index=False)

## Step 2 add acad ranking (manually input PACA data found from 2015), FSM and pupdest
ks4_2016_add = pd.read_csv('schools/brighton/2012-2019/2015-2016_brighton_ks4_completed.csv')
ks4_2016_add['URN'] = ks4_2016_add['URN'].astype(int)

# acad ranking
ks4_2016_add['16_acad_rank'] = ks4_2016_add['gcse_16'].rank(ascending=False)
ks4_2016_add.sort_values('16_acad_rank', ascending=True, inplace=True)

#fsm 
fsm_2016 = pd.read_csv('schools/brighton/fsm/2015-2016_846_cfr.csv')
fsm_2016['URN'] = pd.to_numeric(fsm_2016['URN'], errors='coerce').dropna().astype(int)
ks4_2016_add = pd.merge(ks4_2016_add, fsm_2016[['URN','FSM']], how='left', on='URN')
#replace BACA with 44.7 (2023 value)
ks4_2016_add.loc[ks4_2016_add['URN'] == 136164, 'FSM'] = 44.7
#replace PACA with 25.6 (2023 value)
ks4_2016_add.loc[ks4_2016_add['URN'] == 137063, 'FSM'] = 25.6

#pupdest
pupd_2016 = pd.read_csv('schools/brighton/pupdest/2015-2016_846_pupdest.csv')
pupd_2016['URN'] = pd.to_numeric(pupd_2016['URN'], errors='coerce').dropna()
pupd_2016['SCH_6THPER'] = pupd_2016['SCH_6THPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2016['6TH_COLPER'] = pupd_2016['6TH_COLPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2016['SF_DEST'] = pupd_2016['SCH_6THPER'] + pupd_2016['6TH_COLPER']

ks4_2016_add = pd.merge(ks4_2016_add, pupd_2016[['URN','SF_DEST']], how='left', on='URN')


#get ready to merge
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2016_add['ofsr_values'] = ks4_2016_add['16_ofsr'].map(mapping)

#complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)

ks4_2016_add['16_rank'] = (ks4_2016_add['16_acad_rank'].rank(ascending=True) * 0.3 + 
                           ks4_2016_add['ofsr_values'].rank(ascending=True, method='dense') * 0.3 + 
                           ks4_2016_add['SF_DEST'].rank(ascending=False) * 0.3 + 
                           ks4_2016_add['FSM'].rank(ascending=True) * 0.1
                          )


bins = 3
labels = ['Outstanding', 'Good', 'Poor']


#building dataframe
ks4_2016_add['16_rank_band'] = pd.qcut(ks4_2016_add['16_rank'], q=3 , labels=labels)
ks4_2016_add.rename(columns={'16_rank_band':"2016"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2016_add['LAESTAB'] = ks4_2016_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2016_add[['LAESTAB','2016']], how='left', on='LAESTAB')

# #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

In [48]:
### 2017
### Blatchington M only changes name for 2019 data
### 2015 onwards full proper ranking, BACA and PACA fsm data assume 2023 
#read in csv
brighton_ks4_17 = pd.read_csv("schools/brighton/2012-2019/2016-2017_846_ks4final.csv")
brighton_ks4_17 = brighton_ks4_17[brighton_ks4_17['RECTYPE']== 1]

#extract key columns
### PT5EM_94_17 for 2017
### but essentially Percentage of pupils achieving 5+ A*-C or equivalents including A*-C in both English and mathematics GCSEs
ks4_2017 = brighton_ks4_17[['LEA','ESTAB', 'URN', 'SCHNAME', 'PT5EM_94_17']]
ks4_2017['LEA'] = ks4_2017['LEA'].astype(int) 
ks4_2017['ESTAB'] = ks4_2017['ESTAB'].astype(int)
ks4_2017['LAESTAB'] = ks4_2017['LEA'].astype(str) + ks4_2017['ESTAB'].astype(str)

ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
ks4_2017_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
#int/ str error: corrections
ks4_2017_add['LAESTAB'] = ks4_2017_add['LAESTAB'].astype(str)
ks4_2017_add = pd.merge(ks4_2017_add, ks4_2017[['LAESTAB', 'URN','PT5EM_94_17']], how='left', on='LAESTAB')
#int/ str error: corrections
ks4_2017_add['PT5EM_94_17'] = ks4_2017_add['PT5EM_94_17'].str.rstrip("%").dropna().astype(float)
ks4_2017_add.rename(columns={"PT5EM_94_17":"gcse_17"},inplace=True)
ks4_2017_add.rename(columns={"SCHNAME":"School name"},inplace=True)

# #merge ks4 acad results with  ofsted rating
ks4_2017_add = pd.merge(ks4_2017_add, ofsr_wide[['SCHNAME',2017]], how='left', left_on='School name', right_on='SCHNAME')
ks4_2017_add.rename(columns={2017:"17_ofsr"}, inplace=True)
#replace Blatchington M with Good (from 2013-2022)
ks4_2017_add['URN'] = ks4_2017_add['URN'].astype(int)
ks4_2017_add.loc[ks4_2017_add['URN'] == 114606, '17_ofsr'] = 'Good'
#### export to missing csv to add in ofsr ( no longer needed now)
##ks4_2016_add.to_csv('schools/brighton/2012-2019/2015-2016_brighton_ks4_completed.csv', index=False)

## Step 2 add acad ranking, FSM and pupdest
ks4_2017_add['URN'] = ks4_2017_add['URN'].astype(int)

# acad ranking
ks4_2017_add['17_acad_rank'] = ks4_2017_add['gcse_17'].rank(ascending=False)
ks4_2017_add.sort_values('17_acad_rank', ascending=True, inplace=True)

#fsm 
fsm_2017 = pd.read_csv('schools/brighton/fsm/2016-2017_846_cfr.csv')
fsm_2017['URN'] = pd.to_numeric(fsm_2017['URN'], errors='coerce').dropna().astype(int)
ks4_2017_add = pd.merge(ks4_2017_add, fsm_2017[['URN','FSM']], how='left', on='URN')
#replace BACA with 44.7 (2023 value)
ks4_2017_add.loc[ks4_2017_add['URN'] == 136164, 'FSM'] = 44.7
#replace PACA with 25.6 (2023 value)
ks4_2017_add.loc[ks4_2017_add['URN'] == 137063, 'FSM'] = 25.6

#pupdest
pupd_2017 = pd.read_csv('schools/brighton/pupdest/2016-2017_846_pupdest.csv')
pupd_2017['URN'] = pd.to_numeric(pupd_2017['URN'], errors='coerce').dropna()
pupd_2017['SCH_6THPER'] = pupd_2017['SCH_6THPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2017['6TH_COLPER'] = pupd_2017['6TH_COLPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2017['SF_DEST'] = pupd_2017['SCH_6THPER'] + pupd_2017['6TH_COLPER']

ks4_2017_add = pd.merge(ks4_2017_add, pupd_2017[['URN','SF_DEST']], how='left', on='URN')


#get ready to merge
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2017_add['ofsr_values'] = ks4_2017_add['17_ofsr'].map(mapping)

#complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)

ks4_2017_add['17_rank'] = (ks4_2017_add['17_acad_rank'].rank(ascending=True) * 0.3 + 
                           ks4_2017_add['ofsr_values'].rank(ascending=True, method='dense') * 0.3 + 
                           ks4_2017_add['SF_DEST'].rank(ascending=False) * 0.3 + 
                           ks4_2017_add['FSM'].rank(ascending=True) * 0.1
                          )


bins = 3
labels = ['Outstanding', 'Good', 'Poor']


#building dataframe
ks4_2017_add['17_rank_band'] = pd.qcut(ks4_2017_add['17_rank'], q=3 , labels=labels)
ks4_2017_add.rename(columns={'17_rank_band':"2017"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2017_add['LAESTAB'] = ks4_2017_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2017_add[['LAESTAB','2017']], how='left', on='LAESTAB')

# #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

In [62]:
### 2018
### King's School! added into ks4_hist_df before running this 2018 cell BUT no pupdest fsm data --> IGNORE tentatively
### Blatchington M only changes name for 2019 data
### 2015 onwards full proper ranking, BACA and PACA fsm data assume 2023 
#read in csv
brighton_ks4_18 = pd.read_csv("schools/brighton/2012-2019/2017-2018_846_ks4final.csv")
brighton_ks4_18 = brighton_ks4_18[brighton_ks4_18['RECTYPE']== 1]

#extract key columns
### PT5EM_94 for 2018
### but essentially Percentage of pupils achieving 5+ A*-C or equivalents including A*-C in both English and mathematics GCSEs
ks4_2018 = brighton_ks4_18[['LEA','ESTAB', 'URN', 'SCHNAME', 'PT5EM_94']]
ks4_2018['LEA'] = ks4_2018['LEA'].astype(int) 
ks4_2018['ESTAB'] = ks4_2018['ESTAB'].astype(int)
ks4_2018['LAESTAB'] = ks4_2018['LEA'].astype(str) + ks4_2018['ESTAB'].astype(str)

ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
ks4_2018_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
#int/ str error: corrections
ks4_2018_add['LAESTAB'] = ks4_2018_add['LAESTAB'].astype(str)
ks4_2018_add = pd.merge(ks4_2018_add, ks4_2018[['LAESTAB', 'URN','PT5EM_94']], how='left', on='LAESTAB')
#int/ str error: corrections
ks4_2018_add['PT5EM_94'] = ks4_2018_add['PT5EM_94'].str.rstrip("%").dropna().astype(float)
ks4_2018_add.rename(columns={"PT5EM_94":"gcse_18"},inplace=True)
ks4_2018_add.rename(columns={"SCHNAME":"School name"},inplace=True)

# #merge ks4 acad results with  ofsted rating
ks4_2018_add = pd.merge(ks4_2018_add, ofsr_wide[['SCHNAME',2018]], how='left', left_on='School name', right_on='SCHNAME')
ks4_2018_add.rename(columns={2018:"18_ofsr"}, inplace=True)
#replace Blatchington M with Good (from 2013-2022)
ks4_2018_add['URN'] = ks4_2018_add['URN'].astype(int)
ks4_2018_add.loc[ks4_2018_add['URN'] == 114606, '18_ofsr'] = 'Good'
#### export to missing csv to add in ofsr ( no longer needed now)
##ks4_2016_add.to_csv('schools/brighton/2012-2019/2015-2016_brighton_ks4_completed.csv', index=False)

## Step 2 add acad ranking, FSM and pupdest
# acad ranking
ks4_2018_add['18_acad_rank'] = ks4_2018_add['gcse_18'].rank(ascending=False)
ks4_2018_add.sort_values('18_acad_rank', ascending=True, inplace=True)

#fsm 
fsm_2018 = pd.read_csv('schools/brighton/fsm/2017-2018_846_cfr.csv')
fsm_2018['URN'] = pd.to_numeric(fsm_2018['URN'], errors='coerce').dropna().astype(int)
ks4_2018_add = pd.merge(ks4_2018_add, fsm_2018[['URN','FSM']], how='left', on='URN')
#replace BACA with 44.7 (2023 value)
ks4_2018_add.loc[ks4_2018_add['URN'] == 136164, 'FSM'] = 44.7
#replace PACA with 25.6 (2023 value)
ks4_2018_add.loc[ks4_2018_add['URN'] == 137063, 'FSM'] = 25.6

#pupdest
pupd_2018 = pd.read_csv('schools/brighton/pupdest/2017-2018_846_pupdest.csv')
pupd_2018['URN'] = pd.to_numeric(pupd_2018['URN'], errors='coerce').dropna()
pupd_2018['SCH_6THPER'] = pupd_2018['SCH_6THPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2018['SIXTH_COLPER'] = pupd_2018['SIXTH_COLPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2018['SF_DEST'] = pupd_2018['SCH_6THPER'] + pupd_2018['SIXTH_COLPER']

ks4_2018_add = pd.merge(ks4_2018_add, pupd_2018[['URN','SF_DEST']], how='left', on='URN')


#get ready to merge
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2018_add['ofsr_values'] = ks4_2018_add['18_ofsr'].map(mapping)

#complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)

ks4_2018_add['18_rank'] = (ks4_2018_add['18_acad_rank'].rank(ascending=True) * 0.3 + 
                           ks4_2018_add['ofsr_values'].rank(ascending=True, method='dense') * 0.3 + 
                           ks4_2018_add['SF_DEST'].rank(ascending=False) * 0.3 + 
                           ks4_2018_add['FSM'].rank(ascending=True) * 0.1
                          )


bins = 3
labels = ['Outstanding', 'Good', 'Poor']


#building dataframe
ks4_2018_add['18_rank_band'] = pd.qcut(ks4_2018_add['18_rank'], q=3 , labels=labels)
ks4_2018_add.rename(columns={'18_rank_band':"2018"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2018_add['LAESTAB'] = ks4_2018_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2018_add[['LAESTAB','2018']], how='left', on='LAESTAB')

# #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)

In [64]:
### 2019
### King's School! added into ks4_hist_df before running this 2018 cell BUT no pupdest fsm data --> IGNORE tentatively
### Blatchington M only changes name for 2019 data --> edited ks4_hist_df manually
### 2015 onwards full proper ranking, BACA and PACA fsm data assume 2023 
#read in csv
brighton_ks4_19 = pd.read_csv("schools/brighton/2012-2019/2018-2019_846_ks4final.csv")
brighton_ks4_19 = brighton_ks4_19[brighton_ks4_19['RECTYPE']== 1]

#extract key columns
### PT5EM_94 for 2019
### but essentially Percentage of pupils achieving 5+ A*-C or equivalents including A*-C in both English and mathematics GCSEs
ks4_2019 = brighton_ks4_19[['LEA','ESTAB', 'URN', 'SCHNAME', 'PT5EM_94']]
ks4_2019['LEA'] = ks4_2019['LEA'].astype(int) 
ks4_2019['ESTAB'] = ks4_2019['ESTAB'].astype(int)
ks4_2019['LAESTAB'] = ks4_2019['LEA'].astype(str) + ks4_2019['ESTAB'].astype(str)

ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
ks4_2019_add = ks4_hist_df[['LAESTAB','School name', 'School postcode']]
#int/ str error: corrections
ks4_2019_add['LAESTAB'] = ks4_2019_add['LAESTAB'].astype(str)
ks4_2019_add = pd.merge(ks4_2019_add, ks4_2019[['LAESTAB', 'URN','PT5EM_94']], how='left', on='LAESTAB')
#int/ str error: corrections
ks4_2019_add['PT5EM_94'] = ks4_2019_add['PT5EM_94'].str.rstrip("%").dropna().astype(float)
ks4_2019_add.rename(columns={"PT5EM_94":"gcse_19"},inplace=True)
ks4_2019_add.rename(columns={"SCHNAME":"School name"},inplace=True)

# #merge ks4 acad results with  ofsted rating
ks4_2019_add = pd.merge(ks4_2019_add, ofsr_wide[['SCHNAME',2019]], how='left', left_on='School name', right_on='SCHNAME')
ks4_2019_add.rename(columns={2019:"19_ofsr"}, inplace=True)
#replace Blatchington M with Good (from 2013-2022)
ks4_2019_add['URN'] = ks4_2019_add['URN'].astype(int)
ks4_2019_add.loc[ks4_2019_add['URN'] == 114606, '19_ofsr'] = 'Good'
#### export to missing csv to add in ofsr ( no longer needed now)
##ks4_2016_add.to_csv('schools/brighton/2012-2019/2015-2016_brighton_ks4_completed.csv', index=False)

## Step 2 add acad ranking, FSM and pupdest
# acad ranking
ks4_2019_add['19_acad_rank'] = ks4_2019_add['gcse_19'].rank(ascending=False)
ks4_2019_add.sort_values('19_acad_rank', ascending=True, inplace=True)

#fsm 
fsm_2019 = pd.read_csv('schools/brighton/fsm/2018-2019_846_cfr.csv')
fsm_2019['URN'] = pd.to_numeric(fsm_2019['URN'], errors='coerce').dropna().astype(int)
ks4_2019_add = pd.merge(ks4_2019_add, fsm_2019[['URN','FSM']], how='left', on='URN')
#replace BACA with 44.7 (2023 value)
ks4_2019_add.loc[ks4_2019_add['URN'] == 136164, 'FSM'] = 44.7
#replace PACA with 25.6 (2023 value)
ks4_2019_add.loc[ks4_2019_add['URN'] == 137063, 'FSM'] = 25.6

#pupdest
pupd_2019 = pd.read_csv('schools/brighton/pupdest/2018-2019_846_pupdest.csv')
pupd_2019['URN'] = pd.to_numeric(pupd_2019['URN'], errors='coerce').dropna()
pupd_2019['SCH_6THPER'] = pupd_2019['SCH_6THPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2019['SIXTH_COLPER'] = pupd_2019['SIXTH_COLPER'].str.strip().replace('SUPP', '0').str.rstrip('%').astype(float)
pupd_2019['SF_DEST'] = pupd_2019['SCH_6THPER'] + pupd_2019['SIXTH_COLPER']

ks4_2019_add = pd.merge(ks4_2019_add, pupd_2019[['URN','SF_DEST']], how='left', on='URN')


#get ready to merge
ks4_hist_df = pd.read_csv('schools/brighton/ks4_hist_df.csv')
# Though Grade 3 wasnt changed to Requires Improvement until 2012, Falmer High (BACA's predecessor) has this in 2007 
# due to us using the last recorded Ofsted rating before it closed for prior years (prior years ratings have been cleaned)
mapping = {"Outstanding": 1, "Good": 2, "Satisfactory": 3, "Requires Improvement": 3, "Inadequate":4}
ks4_2019_add['ofsr_values'] = ks4_2019_add['19_ofsr'].map(mapping)

#complete rank
#using ['acad_rank'] and ['OFSR'], we sort the schools into 4 bins (Outstanding, Good, Satisfactory and Poor)

ks4_2019_add['19_rank'] = (ks4_2019_add['19_acad_rank'].rank(ascending=True) * 0.3 + 
                           ks4_2019_add['ofsr_values'].rank(ascending=True, method='dense') * 0.3 + 
                           ks4_2019_add['SF_DEST'].rank(ascending=False) * 0.3 + 
                           ks4_2019_add['FSM'].rank(ascending=True) * 0.1
                          )


bins = 3
labels = ['Outstanding', 'Good', 'Poor']


#building dataframe
ks4_2019_add['19_rank_band'] = pd.qcut(ks4_2019_add['19_rank'], q=3 , labels=labels)
ks4_2019_add.rename(columns={'19_rank_band':"2019"}, inplace=True)

# #left join to ks2_hist_df
ks4_hist_df['LAESTAB'] = ks4_hist_df['LAESTAB'].astype(str)
ks4_2019_add['LAESTAB'] = ks4_2019_add['LAESTAB'].astype(str)
ks4_hist_df = pd.merge(ks4_hist_df, ks4_2019_add[['LAESTAB','2019']], how='left', on='LAESTAB')

# #ks4_hist_df.to_csv('schools/brighton/ks4_hist_df.csv', index=False)